In [ ]:
#####متن اول(Aggregation.txt)

In [ ]:
#####خوشه بندی agglomorative

In [145]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
from sklearn.cluster import AgglomerativeClustering
from scipy.spatial.distance import pdist
link = ["http://cs.joensuu.fi/sipu/datasets/Aggregation.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
values = dataset[["x1", "x2"]].values
distances = pdist(values)
sigma = np.mean(distances)
def gaussian_kernel(x, y, sigma):
    return np.exp(-np.linalg.norm(x - y)**2 / (2 * sigma**2))
num_points = len(values)
weight_matrix = np.zeros((num_points, num_points))
for i in range(num_points):
    for j in range(i + 1, num_points):
        weight_matrix[i, j] = gaussian_kernel(values[i], values[j], sigma)
        weight_matrix[j, i] = weight_matrix[i, j]
agglomerative_clustering = AgglomerativeClustering(n_clusters=None, distance_threshold=1.5, linkage='average')
agglomerative_clustering.fit(values)
def node_distance(linkage_matrix, point1, point2, num_points):
    node1, node2 = point1, point2
    if node1 < num_points and node2 < num_points:
        return 1
    while node1 >= num_points or node2 >= num_points:
        if node1 >= num_points:
            node1 = int(linkage_matrix[node1 - num_points, 0])
        elif node2 >= num_points:
            node2 = int(linkage_matrix[node2 - num_points, 0])
        else:
            if linkage_matrix[node1 - num_points, 0] == linkage_matrix[node2 - num_points, 0]:
                break
            if linkage_matrix[node1 - num_points, 2] < linkage_matrix[node2 - num_points, 2]:
                node1 = int(linkage_matrix[node1 - num_points, 0])
            else:
                node2 = int(linkage_matrix[node2 - num_points, 0])
    return linkage_matrix[node1 - num_points, 2]
def dasgupta_cost(linkage_matrix, weight_matrix):
    num_points = weight_matrix.shape[0]
    total_cost = 0
    for point1 in range(num_points):
        for point2 in range(point1 + 1, num_points):
            if weight_matrix[point1, point2] > 0:
                lca_dist = node_distance(linkage_matrix, point1, point2, num_points)
                total_cost += weight_matrix[point1, point2] * lca_dist
    return total_cost
linkage_matrix = agglomerative_clustering.children_
weight_matrix = weight_matrix
cost = dasgupta_cost(linkage_matrix, weight_matrix)
print("cost_dasgupta", cost)

cost_dasgupta 186366.61059553578


In [ ]:
#### خوشه بندی سلسله مراتبی طیفی برای k=3

In [103]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import pdist
from scipy.linalg import eigh
from scipy.cluster.hierarchy import linkage, fcluster
link= ["http://cs.joensuu.fi/sipu/datasets/Aggregation.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
values = dataset[["x1", "x2"]].values
distances = pdist(values)
sigma = np.mean(distances)
distances = pdist(values)
sigma = np.mean(distances)
def gaussian_kernel(x, y, sigma):
    return np.exp(-np.linalg.norm(x - y)**2 / (2 * sigma**2))
def create_graph(values, k, sigma):
    num_points = len(values)
    knn = NearestNeighbors(n_neighbors=k)
    knn.fit(values)
    distances, indices = knn.kneighbors(values)
    weight_matrix = np.zeros((num_points, num_points))
    for i in range(num_points):
        for j, idx in zip(range(k), indices[i]):
            if i != idx:
                weight_matrix[i, idx] = gaussian_kernel(values[i], values[idx], sigma)
                weight_matrix[idx, i] = weight_matrix[i, idx]
    return weight_matrix
def laplacian_matrix(weight_matrix):
    degree_matrix = np.diag(np.sum(weight_matrix, axis=1))
    laplacian = degree_matrix - weight_matrix
    return laplacian
def second_eigenvector(laplacian):
    eigenvalues, eigenvectors = eigh(laplacian)
    return eigenvectors[:, 1]
def spectral_cut(values, weight_matrix, k, sigma):
    W = create_graph(values, k, sigma)
    L = laplacian_matrix(W)
    second_eig = second_eigenvector(L)
    partition = np.sign(second_eig)
    return partition, second_eig
def hierarchical_spectral_clustering(values, k, sigma):
    weight_matrix = create_graph(values, k, sigma)
    linkage_matrix = linkage(values, method='ward')
    return linkage_matrix, weight_matrix
def node_distance(linkage_matrix, point1, point2, num_points):
    node1, node2 = point1, point2
    if node1 < num_points and node2 < num_points:
        return 1
    while node1 >= num_points or node2 >= num_points:
        if node1 >= num_points:
            node1 = int(linkage_matrix[node1 - num_points, 0])
        elif node2 >= num_points:
            node2 = int(linkage_matrix[node2 - num_points, 0])
        else:
            if linkage_matrix[node1 - num_points, 0] == linkage_matrix[node2 - num_points, 0]:
                break
            if linkage_matrix[node1 - num_points, 2] < linkage_matrix[node2 - num_points, 2]:
                node1 = int(linkage_matrix[node1 - num_points, 0])
            else:
                node2 = int(linkage_matrix[node2 - num_points, 0])
    return linkage_matrix[node1 - num_points, 2]
def dasgupta_cost(linkage_matrix, weight_matrix):
    num_points = weight_matrix.shape[0]
    total_cost = 0
    for point1 in range(num_points):
        for point2 in range(point1 + 1, num_points):
            if weight_matrix[point1, point2] > 0:
                lca_dist = node_distance(linkage_matrix, point1, point2, num_points)
                total_cost += weight_matrix[point1, point2] * lca_dist
    return total_cost
k = 3
linkage_matrix, weight_matrix = hierarchical_spectral_clustering(values, k, sigma)
cost = dasgupta_cost(linkage_matrix, weight_matrix)
print("dasgupta_cost for k=3",cost)


dasgupta_cost for k=3 994.0855011281172


In [ ]:
#### خوشه بندی سلسله مراتبی طیفی برای k=5

In [104]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import pdist
from scipy.linalg import eigh
from scipy.cluster.hierarchy import linkage, fcluster
link= ["http://cs.joensuu.fi/sipu/datasets/Aggregation.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
values = dataset[["x1", "x2"]].values
distances = pdist(values)
sigma = np.mean(distances)
distances = pdist(values)
sigma = np.mean(distances)
def gaussian_kernel(x, y, sigma):
    return np.exp(-np.linalg.norm(x - y)**2 / (2 * sigma**2))
def create_graph(values, k, sigma):
    num_points = len(values)
    knn = NearestNeighbors(n_neighbors=k)
    knn.fit(values)
    distances, indices = knn.kneighbors(values)
    weight_matrix = np.zeros((num_points, num_points))
    for i in range(num_points):
        for j, idx in zip(range(k), indices[i]):
            if i != idx:
                weight_matrix[i, idx] = gaussian_kernel(values[i], values[idx], sigma)
                weight_matrix[idx, i] = weight_matrix[i, idx]
    return weight_matrix
def laplacian_matrix(weight_matrix):
    degree_matrix = np.diag(np.sum(weight_matrix, axis=1))
    laplacian = degree_matrix - weight_matrix
    return laplacian
def second_eigenvector(laplacian):
    eigenvalues, eigenvectors = eigh(laplacian)
    return eigenvectors[:, 1]
def spectral_cut(values, weight_matrix, k, sigma):
    W = create_graph(values, k, sigma)
    L = laplacian_matrix(W)
    second_eig = second_eigenvector(L)
    partition = np.sign(second_eig)
    return partition, second_eig
def hierarchical_spectral_clustering(values, k, sigma):
    weight_matrix = create_graph(values, k, sigma)
    linkage_matrix = linkage(values, method='ward')
    return linkage_matrix, weight_matrix
def node_distance(linkage_matrix, point1, point2, num_points):
    node1, node2 = point1, point2
    if node1 < num_points and node2 < num_points:
        return 1
    while node1 >= num_points or node2 >= num_points:
        if node1 >= num_points:
            node1 = int(linkage_matrix[node1 - num_points, 0])
        elif node2 >= num_points:
            node2 = int(linkage_matrix[node2 - num_points, 0])
        else:
            if linkage_matrix[node1 - num_points, 0] == linkage_matrix[node2 - num_points, 0]:
                break
            if linkage_matrix[node1 - num_points, 2] < linkage_matrix[node2 - num_points, 2]:
                node1 = int(linkage_matrix[node1 - num_points, 0])
            else:
                node2 = int(linkage_matrix[node2 - num_points, 0])
    return linkage_matrix[node1 - num_points, 2]
def dasgupta_cost(linkage_matrix, weight_matrix):
    num_points = weight_matrix.shape[0]
    total_cost = 0
    for point1 in range(num_points):
        for point2 in range(point1 + 1, num_points):
            if weight_matrix[point1, point2] > 0:
                lca_dist = node_distance(linkage_matrix, point1, point2, num_points)
                total_cost += weight_matrix[point1, point2] * lca_dist
    return total_cost
k =5
linkage_matrix, weight_matrix = hierarchical_spectral_clustering(values, k, sigma)
cost = dasgupta_cost(linkage_matrix, weight_matrix)
print("dasgupta_cost for k=5",cost)

dasgupta_cost for k=5 1834.5087769683992


In [ ]:
#### خوشه بندی سلسله مراتبی طیفی برای k=10

In [105]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import pdist
from scipy.linalg import eigh
from scipy.cluster.hierarchy import linkage, fcluster
link= ["http://cs.joensuu.fi/sipu/datasets/Aggregation.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
values = dataset[["x1", "x2"]].values
distances = pdist(values)
sigma = np.mean(distances)
distances = pdist(values)
sigma = np.mean(distances)
def gaussian_kernel(x, y, sigma):
    return np.exp(-np.linalg.norm(x - y)**2 / (2 * sigma**2))
def create_graph(values, k, sigma):
    num_points = len(values)
    knn = NearestNeighbors(n_neighbors=k)
    knn.fit(values)
    distances, indices = knn.kneighbors(values)
    weight_matrix = np.zeros((num_points, num_points))
    for i in range(num_points):
        for j, idx in zip(range(k), indices[i]):
            if i != idx:
                weight_matrix[i, idx] = gaussian_kernel(values[i], values[idx], sigma)
                weight_matrix[idx, i] = weight_matrix[i, idx]
    return weight_matrix
def laplacian_matrix(weight_matrix):
    degree_matrix = np.diag(np.sum(weight_matrix, axis=1))
    laplacian = degree_matrix - weight_matrix
    return laplacian
def second_eigenvector(laplacian):
    eigenvalues, eigenvectors = eigh(laplacian)
    return eigenvectors[:, 1]
def spectral_cut(values, weight_matrix, k, sigma):
    W = create_graph(values, k, sigma)
    L = laplacian_matrix(W)
    second_eig = second_eigenvector(L)
    partition = np.sign(second_eig)
    return partition, second_eig
def hierarchical_spectral_clustering(values, k, sigma):
    weight_matrix = create_graph(values, k, sigma)
    linkage_matrix = linkage(values, method='ward')
    return linkage_matrix, weight_matrix
def node_distance(linkage_matrix, point1, point2, num_points):
    node1, node2 = point1, point2
    if node1 < num_points and node2 < num_points:
        return 1
    while node1 >= num_points or node2 >= num_points:
        if node1 >= num_points:
            node1 = int(linkage_matrix[node1 - num_points, 0])
        elif node2 >= num_points:
            node2 = int(linkage_matrix[node2 - num_points, 0])
        else:
            if linkage_matrix[node1 - num_points, 0] == linkage_matrix[node2 - num_points, 0]:
                break
            if linkage_matrix[node1 - num_points, 2] < linkage_matrix[node2 - num_points, 2]:
                node1 = int(linkage_matrix[node1 - num_points, 0])
            else:
                node2 = int(linkage_matrix[node2 - num_points, 0])
    return linkage_matrix[node1 - num_points, 2]
def dasgupta_cost(linkage_matrix, weight_matrix):
    num_points = weight_matrix.shape[0]
    total_cost = 0
    for point1 in range(num_points):
        for point2 in range(point1 + 1, num_points):
            if weight_matrix[point1, point2] > 0:
                lca_dist = node_distance(linkage_matrix, point1, point2, num_points)
                total_cost += weight_matrix[point1, point2] * lca_dist
    return total_cost
k =10
linkage_matrix, weight_matrix = hierarchical_spectral_clustering(values, k, sigma)
cost = dasgupta_cost(linkage_matrix, weight_matrix)
print("dasgupta_cost for k=10",cost)

dasgupta_cost for k=10 4015.878531092625


In [ ]:
####### جدول مقایسه

In [118]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import AgglomerativeClustering
from scipy.spatial.distance import pdist
from scipy.linalg import eigh
from scipy.cluster.hierarchy import linkage
link = ["http://cs.joensuu.fi/sipu/datasets/Aggregation.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
values = dataset[["x1", "x2"]].values
distances = pdist(values)
sigma = np.mean(distances)
def gaussian_kernel(x, y, sigma):
    return np.exp(-np.linalg.norm(x - y)**2 / (2 * sigma**2))
def create_spectral_graph(values, k, sigma):
    num_points = len(values)
    knn = NearestNeighbors(n_neighbors=k)
    knn.fit(values)
    distances, indices = knn.kneighbors(values)
    weight_matrix = np.zeros((num_points, num_points))
    for i in range(num_points):
        for j, idx in zip(range(k), indices[i]):
            if i != idx:
                weight_matrix[i, idx] = gaussian_kernel(values[i], values[idx], sigma)
                weight_matrix[idx, i] = weight_matrix[i, idx]
    return weight_matrix
def spectral_laplacian_matrix(weight_matrix):
    degree_matrix = np.diag(np.sum(weight_matrix, axis=1))
    laplacian = degree_matrix - weight_matrix
    return laplacian
def spectral_second_eigenvector(laplacian):
    eigenvalues, eigenvectors = eigh(laplacian)
    return eigenvectors[:, 1]
def spectral_dasgupta_cost(linkage_matrix, weight_matrix):
    num_points = weight_matrix.shape[0]
    total_cost = 0
    for point1 in range(num_points):
        for point2 in range(point1 + 1, num_points):
            if weight_matrix[point1, point2] > 0:
                lca_dist = node_distance(linkage_matrix, point1, point2, num_points)
                total_cost += weight_matrix[point1, point2] * lca_dist
    return total_cost
def create_agglomerative_graph(values, sigma):
    num_points = len(values)
    weight_matrix = np.zeros((num_points, num_points))
    for i in range(num_points):
        for j in range(i + 1, num_points):
            weight_matrix[i, j] = gaussian_kernel(values[i], values[j], sigma)
            weight_matrix[j, i] = weight_matrix[i, j]
    return weight_matrix
agglomerative_clustering = AgglomerativeClustering(n_clusters=None, distance_threshold=1.5, linkage='average')
agglomerative_clustering.fit(values)
dataset["new_labels"] = agglomerative_clustering.labels_
linkage_matrix_agglomerative = agglomerative_clustering.children_
weight_matrix_agglomerative = create_agglomerative_graph(values, sigma)
cost_agglomerative = spectral_dasgupta_cost(linkage_matrix_agglomerative, weight_matrix_agglomerative)
k_values = [3, 5, 10]
cost_spectral_values = []
for k in k_values:
    weight_matrix_spectral = create_spectral_graph(values, k, sigma)
    laplacian_spectral = spectral_laplacian_matrix(weight_matrix_spectral)
    second_eig_spectral = spectral_second_eigenvector(laplacian_spectral)
    linkage_matrix_spectral = linkage(values, method='ward')
    cost_spectral = spectral_dasgupta_cost(linkage_matrix_spectral, weight_matrix_spectral)
    cost_spectral_values.append(cost_spectral)
data = {"Method": ["Spectral Clustering (k=3)", "Spectral Clustering (k=5)", "Spectral Clustering (k=10)", "Agglomerative Clustering"],
    "Dasgupta Cost": cost_spectral_values + [cost_agglomerative]}
df_result = pd.DataFrame(data)
print(df_result)

                       Method  Dasgupta Cost
0   Spectral Clustering (k=3)     994.085501
1   Spectral Clustering (k=5)    1834.508777
2  Spectral Clustering (k=10)    4015.878531
3    Agglomerative Clustering  186366.610596


In [ ]:
############ متن دوم(Compound.txt)

In [144]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
from sklearn.cluster import AgglomerativeClustering
from scipy.spatial.distance import pdist
link = ["https://cs.joensuu.fi/sipu/datasets/Compound.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
values = dataset[["x1", "x2"]].values
distances = pdist(values)
sigma = np.mean(distances)
def gaussian_kernel(x, y, sigma):
    return np.exp(-np.linalg.norm(x - y)**2 / (2 * sigma**2))
num_points = len(values)
weight_matrix = np.zeros((num_points, num_points))
for i in range(num_points):
    for j in range(i + 1, num_points):
        weight_matrix[i, j] = gaussian_kernel(values[i], values[j], sigma)
        weight_matrix[j, i] = weight_matrix[i, j]
agglomerative_clustering = AgglomerativeClustering(n_clusters=None, distance_threshold=1.5, linkage='average')
agglomerative_clustering.fit(values)
def node_distance(linkage_matrix, point1, point2, num_points):
    node1, node2 = point1, point2
    if node1 < num_points and node2 < num_points:
        return 1
    while node1 >= num_points or node2 >= num_points:
        if node1 >= num_points:
            node1 = int(linkage_matrix[node1 - num_points, 0])
        elif node2 >= num_points:
            node2 = int(linkage_matrix[node2 - num_points, 0])
        else:
            if linkage_matrix[node1 - num_points, 0] == linkage_matrix[node2 - num_points, 0]:
                break
            if linkage_matrix[node1 - num_points, 2] < linkage_matrix[node2 - num_points, 2]:
                node1 = int(linkage_matrix[node1 - num_points, 0])
            else:
                node2 = int(linkage_matrix[node2 - num_points, 0])
    return linkage_matrix[node1 - num_points, 2]
def dasgupta_cost(linkage_matrix, weight_matrix):
    num_points = weight_matrix.shape[0]
    total_cost = 0
    for point1 in range(num_points):
        for point2 in range(point1 + 1, num_points):
            if weight_matrix[point1, point2] > 0:
                lca_dist = node_distance(linkage_matrix, point1, point2, num_points)
                total_cost += weight_matrix[point1, point2] * lca_dist
    return total_cost
linkage_matrix = agglomerative_clustering.children_
weight_matrix = weight_matrix
cost = dasgupta_cost(linkage_matrix, weight_matrix)
print("cost_dasgupta", cost)

cost_dasgupta 48099.511786097166


In [ ]:
#### خوشه بندی سلسله مراتبی طیفی برای k=3

In [107]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import pdist
from scipy.linalg import eigh
from scipy.cluster.hierarchy import linkage, fcluster
link= ["https://cs.joensuu.fi/sipu/datasets/Compound.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
values = dataset[["x1", "x2"]].values
distances = pdist(values)
sigma = np.mean(distances)
distances = pdist(values)
sigma = np.mean(distances)
def gaussian_kernel(x, y, sigma):
    return np.exp(-np.linalg.norm(x - y)**2 / (2 * sigma**2))
def create_graph(values, k, sigma):
    num_points = len(values)
    knn = NearestNeighbors(n_neighbors=k)
    knn.fit(values)
    distances, indices = knn.kneighbors(values)
    weight_matrix = np.zeros((num_points, num_points))
    for i in range(num_points):
        for j, idx in zip(range(k), indices[i]):
            if i != idx:
                weight_matrix[i, idx] = gaussian_kernel(values[i], values[idx], sigma)
                weight_matrix[idx, i] = weight_matrix[i, idx]
    return weight_matrix
def laplacian_matrix(weight_matrix):
    degree_matrix = np.diag(np.sum(weight_matrix, axis=1))
    laplacian = degree_matrix - weight_matrix
    return laplacian
def second_eigenvector(laplacian):
    eigenvalues, eigenvectors = eigh(laplacian)
    return eigenvectors[:, 1]
def spectral_cut(values, weight_matrix, k, sigma):
    W = create_graph(values, k, sigma)
    L = laplacian_matrix(W)
    second_eig = second_eigenvector(L)
    partition = np.sign(second_eig)
    return partition, second_eig
def hierarchical_spectral_clustering(values, k, sigma):
    weight_matrix = create_graph(values, k, sigma)
    linkage_matrix = linkage(values, method='ward')
    return linkage_matrix, weight_matrix
def node_distance(linkage_matrix, point1, point2, num_points):
    node1, node2 = point1, point2
    if node1 < num_points and node2 < num_points:
        return 1
    while node1 >= num_points or node2 >= num_points:
        if node1 >= num_points:
            node1 = int(linkage_matrix[node1 - num_points, 0])
        elif node2 >= num_points:
            node2 = int(linkage_matrix[node2 - num_points, 0])
        else:
            if linkage_matrix[node1 - num_points, 0] == linkage_matrix[node2 - num_points, 0]:
                break
            if linkage_matrix[node1 - num_points, 2] < linkage_matrix[node2 - num_points, 2]:
                node1 = int(linkage_matrix[node1 - num_points, 0])
            else:
                node2 = int(linkage_matrix[node2 - num_points, 0])
    return linkage_matrix[node1 - num_points, 2]
def dasgupta_cost(linkage_matrix, weight_matrix):
    num_points = weight_matrix.shape[0]
    total_cost = 0
    for point1 in range(num_points):
        for point2 in range(point1 + 1, num_points):
            if weight_matrix[point1, point2] > 0:
                lca_dist = node_distance(linkage_matrix, point1, point2, num_points)
                total_cost += weight_matrix[point1, point2] * lca_dist
    return total_cost
k = 3
linkage_matrix, weight_matrix = hierarchical_spectral_clustering(values, k, sigma)
cost = dasgupta_cost(linkage_matrix, weight_matrix)
print("dasgupta_cost for k=3",cost)

dasgupta_cost for k=3 531.3772012032567


In [ ]:
#### خوشه بندی سلسله مراتبی طیفی برای k=5

In [108]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import pdist
from scipy.linalg import eigh
from scipy.cluster.hierarchy import linkage, fcluster
link= ["https://cs.joensuu.fi/sipu/datasets/Compound.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
values = dataset[["x1", "x2"]].values
distances = pdist(values)
sigma = np.mean(distances)
distances = pdist(values)
sigma = np.mean(distances)
def gaussian_kernel(x, y, sigma):
    return np.exp(-np.linalg.norm(x - y)**2 / (2 * sigma**2))
def create_graph(values, k, sigma):
    num_points = len(values)
    knn = NearestNeighbors(n_neighbors=k)
    knn.fit(values)
    distances, indices = knn.kneighbors(values)
    weight_matrix = np.zeros((num_points, num_points))
    for i in range(num_points):
        for j, idx in zip(range(k), indices[i]):
            if i != idx:
                weight_matrix[i, idx] = gaussian_kernel(values[i], values[idx], sigma)
                weight_matrix[idx, i] = weight_matrix[i, idx]
    return weight_matrix
def laplacian_matrix(weight_matrix):
    degree_matrix = np.diag(np.sum(weight_matrix, axis=1))
    laplacian = degree_matrix - weight_matrix
    return laplacian
def second_eigenvector(laplacian):
    eigenvalues, eigenvectors = eigh(laplacian)
    return eigenvectors[:, 1]
def spectral_cut(values, weight_matrix, k, sigma):
    W = create_graph(values, k, sigma)
    L = laplacian_matrix(W)
    second_eig = second_eigenvector(L)
    partition = np.sign(second_eig)
    return partition, second_eig
def hierarchical_spectral_clustering(values, k, sigma):
    weight_matrix = create_graph(values, k, sigma)
    linkage_matrix = linkage(values, method='ward')
    return linkage_matrix, weight_matrix
def node_distance(linkage_matrix, point1, point2, num_points):
    node1, node2 = point1, point2
    if node1 < num_points and node2 < num_points:
        return 1
    while node1 >= num_points or node2 >= num_points:
        if node1 >= num_points:
            node1 = int(linkage_matrix[node1 - num_points, 0])
        elif node2 >= num_points:
            node2 = int(linkage_matrix[node2 - num_points, 0])
        else:
            if linkage_matrix[node1 - num_points, 0] == linkage_matrix[node2 - num_points, 0]:
                break
            if linkage_matrix[node1 - num_points, 2] < linkage_matrix[node2 - num_points, 2]:
                node1 = int(linkage_matrix[node1 - num_points, 0])
            else:
                node2 = int(linkage_matrix[node2 - num_points, 0])
    return linkage_matrix[node1 - num_points, 2]
def dasgupta_cost(linkage_matrix, weight_matrix):
    num_points = weight_matrix.shape[0]
    total_cost = 0
    for point1 in range(num_points):
        for point2 in range(point1 + 1, num_points):
            if weight_matrix[point1, point2] > 0:
                lca_dist = node_distance(linkage_matrix, point1, point2, num_points)
                total_cost += weight_matrix[point1, point2] * lca_dist
    return total_cost
k =5
linkage_matrix, weight_matrix = hierarchical_spectral_clustering(values, k, sigma)
cost = dasgupta_cost(linkage_matrix, weight_matrix)
print("dasgupta_cost for k=5",cost)

dasgupta_cost for k=5 954.3126286591983


In [ ]:
#### خوشه بندی سلسله مراتبی طیفی برای k=10

In [109]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import pdist
from scipy.linalg import eigh
from scipy.cluster.hierarchy import linkage, fcluster
link= ["https://cs.joensuu.fi/sipu/datasets/Compound.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
values = dataset[["x1", "x2"]].values
distances = pdist(values)
sigma = np.mean(distances)
distances = pdist(values)
sigma = np.mean(distances)
def gaussian_kernel(x, y, sigma):
    return np.exp(-np.linalg.norm(x - y)**2 / (2 * sigma**2))
def create_graph(values, k, sigma):
    num_points = len(values)
    knn = NearestNeighbors(n_neighbors=k)
    knn.fit(values)
    distances, indices = knn.kneighbors(values)
    weight_matrix = np.zeros((num_points, num_points))
    for i in range(num_points):
        for j, idx in zip(range(k), indices[i]):
            if i != idx:
                weight_matrix[i, idx] = gaussian_kernel(values[i], values[idx], sigma)
                weight_matrix[idx, i] = weight_matrix[i, idx]
    return weight_matrix
def laplacian_matrix(weight_matrix):
    degree_matrix = np.diag(np.sum(weight_matrix, axis=1))
    laplacian = degree_matrix - weight_matrix
    return laplacian
def second_eigenvector(laplacian):
    eigenvalues, eigenvectors = eigh(laplacian)
    return eigenvectors[:, 1]
def spectral_cut(values, weight_matrix, k, sigma):
    W = create_graph(values, k, sigma)
    L = laplacian_matrix(W)
    second_eig = second_eigenvector(L)
    partition = np.sign(second_eig)
    return partition, second_eig
def hierarchical_spectral_clustering(values, k, sigma):
    weight_matrix = create_graph(values, k, sigma)
    linkage_matrix = linkage(values, method='ward')
    return linkage_matrix, weight_matrix
def node_distance(linkage_matrix, point1, point2, num_points):
    node1, node2 = point1, point2
    if node1 < num_points and node2 < num_points:
        return 1
    while node1 >= num_points or node2 >= num_points:
        if node1 >= num_points:
            node1 = int(linkage_matrix[node1 - num_points, 0])
        elif node2 >= num_points:
            node2 = int(linkage_matrix[node2 - num_points, 0])
        else:
            if linkage_matrix[node1 - num_points, 0] == linkage_matrix[node2 - num_points, 0]:
                break
            if linkage_matrix[node1 - num_points, 2] < linkage_matrix[node2 - num_points, 2]:
                node1 = int(linkage_matrix[node1 - num_points, 0])
            else:
                node2 = int(linkage_matrix[node2 - num_points, 0])
    return linkage_matrix[node1 - num_points, 2]
def dasgupta_cost(linkage_matrix, weight_matrix):
    num_points = weight_matrix.shape[0]
    total_cost = 0
    for point1 in range(num_points):
        for point2 in range(point1 + 1, num_points):
            if weight_matrix[point1, point2] > 0:
                lca_dist = node_distance(linkage_matrix, point1, point2, num_points)
                total_cost += weight_matrix[point1, point2] * lca_dist
    return total_cost
k =10
linkage_matrix, weight_matrix = hierarchical_spectral_clustering(values, k, sigma)
cost = dasgupta_cost(linkage_matrix, weight_matrix)
print("dasgupta_cost for k=10",cost)

dasgupta_cost for k=10 2164.780801966074


In [ ]:
####### جدول مقایسه

In [117]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import AgglomerativeClustering
from scipy.spatial.distance import pdist
from scipy.linalg import eigh
from scipy.cluster.hierarchy import linkage
link = ["https://cs.joensuu.fi/sipu/datasets/Compound.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
values = dataset[["x1", "x2"]].values
distances = pdist(values)
sigma = np.mean(distances)
def gaussian_kernel(x, y, sigma):
    return np.exp(-np.linalg.norm(x - y)**2 / (2 * sigma**2))
def create_spectral_graph(values, k, sigma):
    num_points = len(values)
    knn = NearestNeighbors(n_neighbors=k)
    knn.fit(values)
    distances, indices = knn.kneighbors(values)
    weight_matrix = np.zeros((num_points, num_points))
    for i in range(num_points):
        for j, idx in zip(range(k), indices[i]):
            if i != idx:
                weight_matrix[i, idx] = gaussian_kernel(values[i], values[idx], sigma)
                weight_matrix[idx, i] = weight_matrix[i, idx]
    return weight_matrix
def spectral_laplacian_matrix(weight_matrix):
    degree_matrix = np.diag(np.sum(weight_matrix, axis=1))
    laplacian = degree_matrix - weight_matrix
    return laplacian
def spectral_second_eigenvector(laplacian):
    eigenvalues, eigenvectors = eigh(laplacian)
    return eigenvectors[:, 1]
def spectral_dasgupta_cost(linkage_matrix, weight_matrix):
    num_points = weight_matrix.shape[0]
    total_cost = 0
    for point1 in range(num_points):
        for point2 in range(point1 + 1, num_points):
            if weight_matrix[point1, point2] > 0:
                lca_dist = node_distance(linkage_matrix, point1, point2, num_points)
                total_cost += weight_matrix[point1, point2] * lca_dist
    return total_cost
def create_agglomerative_graph(values, sigma):
    num_points = len(values)
    weight_matrix = np.zeros((num_points, num_points))
    for i in range(num_points):
        for j in range(i + 1, num_points):
            weight_matrix[i, j] = gaussian_kernel(values[i], values[j], sigma)
            weight_matrix[j, i] = weight_matrix[i, j]
    return weight_matrix
agglomerative_clustering = AgglomerativeClustering(n_clusters=None, distance_threshold=1.5, linkage='average')
agglomerative_clustering.fit(values)
dataset["new_labels"] = agglomerative_clustering.labels_
linkage_matrix_agglomerative = agglomerative_clustering.children_
weight_matrix_agglomerative = create_agglomerative_graph(values, sigma)
cost_agglomerative = spectral_dasgupta_cost(linkage_matrix_agglomerative, weight_matrix_agglomerative)
k_values = [3, 5, 10]
cost_spectral_values = []
for k in k_values:
    weight_matrix_spectral = create_spectral_graph(values, k, sigma)
    laplacian_spectral = spectral_laplacian_matrix(weight_matrix_spectral)
    second_eig_spectral = spectral_second_eigenvector(laplacian_spectral)
    linkage_matrix_spectral = linkage(values, method='ward')
    cost_spectral = spectral_dasgupta_cost(linkage_matrix_spectral, weight_matrix_spectral)
    cost_spectral_values.append(cost_spectral)
data = {"Method": ["Spectral Clustering (k=3)", "Spectral Clustering (k=5)", "Spectral Clustering (k=10)", "Agglomerative Clustering"],
    "Dasgupta Cost": cost_spectral_values + [cost_agglomerative]}
df_result = pd.DataFrame(data)
print(df_result)

                       Method  Dasgupta Cost
0   Spectral Clustering (k=3)     531.377201
1   Spectral Clustering (k=5)     954.312629
2  Spectral Clustering (k=10)    2164.780802
3    Agglomerative Clustering   48099.511786


In [ ]:
###### متن سوم(pathbased.txt)

In [ ]:
#####خوشه بندی agglomorative

In [146]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
from sklearn.cluster import AgglomerativeClustering
from scipy.spatial.distance import pdist
link = ["https://cs.joensuu.fi/sipu/datasets/pathbased.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
values = dataset[["x1", "x2"]].values
distances = pdist(values)
sigma = np.mean(distances)
def gaussian_kernel(x, y, sigma):
    return np.exp(-np.linalg.norm(x - y)**2 / (2 * sigma**2))
num_points = len(values)
weight_matrix = np.zeros((num_points, num_points))
for i in range(num_points):
    for j in range(i + 1, num_points):
        weight_matrix[i, j] = gaussian_kernel(values[i], values[j], sigma)
        weight_matrix[j, i] = weight_matrix[i, j]
agglomerative_clustering = AgglomerativeClustering(n_clusters=None, distance_threshold=1.5, linkage='average')
agglomerative_clustering.fit(values)
def node_distance(linkage_matrix, point1, point2, num_points):
    node1, node2 = point1, point2
    if node1 < num_points and node2 < num_points:
        return 1
    while node1 >= num_points or node2 >= num_points:
        if node1 >= num_points:
            node1 = int(linkage_matrix[node1 - num_points, 0])
        elif node2 >= num_points:
            node2 = int(linkage_matrix[node2 - num_points, 0])
        else:
            if linkage_matrix[node1 - num_points, 0] == linkage_matrix[node2 - num_points, 0]:
                break
            if linkage_matrix[node1 - num_points, 2] < linkage_matrix[node2 - num_points, 2]:
                node1 = int(linkage_matrix[node1 - num_points, 0])
            else:
                node2 = int(linkage_matrix[node2 - num_points, 0])
    return linkage_matrix[node1 - num_points, 2]
def dasgupta_cost(linkage_matrix, weight_matrix):
    num_points = weight_matrix.shape[0]
    total_cost = 0
    for point1 in range(num_points):
        for point2 in range(point1 + 1, num_points):
            if weight_matrix[point1, point2] > 0:
                lca_dist = node_distance(linkage_matrix, point1, point2, num_points)
                total_cost += weight_matrix[point1, point2] * lca_dist
    return total_cost
linkage_matrix = agglomerative_clustering.children_
weight_matrix = weight_matrix
cost = dasgupta_cost(linkage_matrix, weight_matrix)
print("cost_dasgupta", cost)

cost_dasgupta 27067.34238239806


In [ ]:
#### خوشه بندی سلسله مراتبی طیفی برای k=3

In [110]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import pdist
from scipy.linalg import eigh
from scipy.cluster.hierarchy import linkage, fcluster
link= ["https://cs.joensuu.fi/sipu/datasets/pathbased.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
values = dataset[["x1", "x2"]].values
distances = pdist(values)
sigma = np.mean(distances)
distances = pdist(values)
sigma = np.mean(distances)
def gaussian_kernel(x, y, sigma):
    return np.exp(-np.linalg.norm(x - y)**2 / (2 * sigma**2))
def create_graph(values, k, sigma):
    num_points = len(values)
    knn = NearestNeighbors(n_neighbors=k)
    knn.fit(values)
    distances, indices = knn.kneighbors(values)
    weight_matrix = np.zeros((num_points, num_points))
    for i in range(num_points):
        for j, idx in zip(range(k), indices[i]):
            if i != idx:
                weight_matrix[i, idx] = gaussian_kernel(values[i], values[idx], sigma)
                weight_matrix[idx, i] = weight_matrix[i, idx]
    return weight_matrix
def laplacian_matrix(weight_matrix):
    degree_matrix = np.diag(np.sum(weight_matrix, axis=1))
    laplacian = degree_matrix - weight_matrix
    return laplacian
def second_eigenvector(laplacian):
    eigenvalues, eigenvectors = eigh(laplacian)
    return eigenvectors[:, 1]
def spectral_cut(values, weight_matrix, k, sigma):
    W = create_graph(values, k, sigma)
    L = laplacian_matrix(W)
    second_eig = second_eigenvector(L)
    partition = np.sign(second_eig)
    return partition, second_eig
def hierarchical_spectral_clustering(values, k, sigma):
    weight_matrix = create_graph(values, k, sigma)
    linkage_matrix = linkage(values, method='ward')
    return linkage_matrix, weight_matrix
def node_distance(linkage_matrix, point1, point2, num_points):
    node1, node2 = point1, point2
    if node1 < num_points and node2 < num_points:
        return 1
    while node1 >= num_points or node2 >= num_points:
        if node1 >= num_points:
            node1 = int(linkage_matrix[node1 - num_points, 0])
        elif node2 >= num_points:
            node2 = int(linkage_matrix[node2 - num_points, 0])
        else:
            if linkage_matrix[node1 - num_points, 0] == linkage_matrix[node2 - num_points, 0]:
                break
            if linkage_matrix[node1 - num_points, 2] < linkage_matrix[node2 - num_points, 2]:
                node1 = int(linkage_matrix[node1 - num_points, 0])
            else:
                node2 = int(linkage_matrix[node2 - num_points, 0])
    return linkage_matrix[node1 - num_points, 2]
def dasgupta_cost(linkage_matrix, weight_matrix):
    num_points = weight_matrix.shape[0]
    total_cost = 0
    for point1 in range(num_points):
        for point2 in range(point1 + 1, num_points):
            if weight_matrix[point1, point2] > 0:
                lca_dist = node_distance(linkage_matrix, point1, point2, num_points)
                total_cost += weight_matrix[point1, point2] * lca_dist
    return total_cost
k =3
linkage_matrix, weight_matrix = hierarchical_spectral_clustering(values, k, sigma)
cost = dasgupta_cost(linkage_matrix, weight_matrix)
print("dasgupta_cost for k=3",cost)

dasgupta_cost for k=3 383.8850705514417


In [ ]:
#### خوشه بندی سلسله مراتبی طیفی برای k=5

In [112]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import pdist
from scipy.linalg import eigh
from scipy.cluster.hierarchy import linkage, fcluster
link= ["https://cs.joensuu.fi/sipu/datasets/pathbased.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
values = dataset[["x1", "x2"]].values
distances = pdist(values)
sigma = np.mean(distances)
distances = pdist(values)
sigma = np.mean(distances)
def gaussian_kernel(x, y, sigma):
    return np.exp(-np.linalg.norm(x - y)**2 / (2 * sigma**2))
def create_graph(values, k, sigma):
    num_points = len(values)
    knn = NearestNeighbors(n_neighbors=k)
    knn.fit(values)
    distances, indices = knn.kneighbors(values)
    weight_matrix = np.zeros((num_points, num_points))
    for i in range(num_points):
        for j, idx in zip(range(k), indices[i]):
            if i != idx:
                weight_matrix[i, idx] = gaussian_kernel(values[i], values[idx], sigma)
                weight_matrix[idx, i] = weight_matrix[i, idx]
    return weight_matrix
def laplacian_matrix(weight_matrix):
    degree_matrix = np.diag(np.sum(weight_matrix, axis=1))
    laplacian = degree_matrix - weight_matrix
    return laplacian
def second_eigenvector(laplacian):
    eigenvalues, eigenvectors = eigh(laplacian)
    return eigenvectors[:, 1]
def spectral_cut(values, weight_matrix, k, sigma):
    W = create_graph(values, k, sigma)
    L = laplacian_matrix(W)
    second_eig = second_eigenvector(L)
    partition = np.sign(second_eig)
    return partition, second_eig
def hierarchical_spectral_clustering(values, k, sigma):
    weight_matrix = create_graph(values, k, sigma)
    linkage_matrix = linkage(values, method='ward')
    return linkage_matrix, weight_matrix
def node_distance(linkage_matrix, point1, point2, num_points):
    node1, node2 = point1, point2
    if node1 < num_points and node2 < num_points:
        return 1
    while node1 >= num_points or node2 >= num_points:
        if node1 >= num_points:
            node1 = int(linkage_matrix[node1 - num_points, 0])
        elif node2 >= num_points:
            node2 = int(linkage_matrix[node2 - num_points, 0])
        else:
            if linkage_matrix[node1 - num_points, 0] == linkage_matrix[node2 - num_points, 0]:
                break
            if linkage_matrix[node1 - num_points, 2] < linkage_matrix[node2 - num_points, 2]:
                node1 = int(linkage_matrix[node1 - num_points, 0])
            else:
                node2 = int(linkage_matrix[node2 - num_points, 0])
    return linkage_matrix[node1 - num_points, 2]
def dasgupta_cost(linkage_matrix, weight_matrix):
    num_points = weight_matrix.shape[0]
    total_cost = 0
    for point1 in range(num_points):
        for point2 in range(point1 + 1, num_points):
            if weight_matrix[point1, point2] > 0:
                lca_dist = node_distance(linkage_matrix, point1, point2, num_points)
                total_cost += weight_matrix[point1, point2] * lca_dist
    return total_cost
k =5
linkage_matrix, weight_matrix = hierarchical_spectral_clustering(values, k, sigma)
cost = dasgupta_cost(linkage_matrix, weight_matrix)
print("dasgupta_cost for k=5",cost)

dasgupta_cost for k=5 727.4555049320236


In [ ]:
#### خوشه بندی سلسله مراتبی طیفی برای k=10

In [113]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import pdist
from scipy.linalg import eigh
from scipy.cluster.hierarchy import linkage, fcluster
link= ["https://cs.joensuu.fi/sipu/datasets/pathbased.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
values = dataset[["x1", "x2"]].values
distances = pdist(values)
sigma = np.mean(distances)
distances = pdist(values)
sigma = np.mean(distances)
def gaussian_kernel(x, y, sigma):
    return np.exp(-np.linalg.norm(x - y)**2 / (2 * sigma**2))
def create_graph(values, k, sigma):
    num_points = len(values)
    knn = NearestNeighbors(n_neighbors=k)
    knn.fit(values)
    distances, indices = knn.kneighbors(values)
    weight_matrix = np.zeros((num_points, num_points))
    for i in range(num_points):
        for j, idx in zip(range(k), indices[i]):
            if i != idx:
                weight_matrix[i, idx] = gaussian_kernel(values[i], values[idx], sigma)
                weight_matrix[idx, i] = weight_matrix[i, idx]
    return weight_matrix
def laplacian_matrix(weight_matrix):
    degree_matrix = np.diag(np.sum(weight_matrix, axis=1))
    laplacian = degree_matrix - weight_matrix
    return laplacian
def second_eigenvector(laplacian):
    eigenvalues, eigenvectors = eigh(laplacian)
    return eigenvectors[:, 1]
def spectral_cut(values, weight_matrix, k, sigma):
    W = create_graph(values, k, sigma)
    L = laplacian_matrix(W)
    second_eig = second_eigenvector(L)
    partition = np.sign(second_eig)
    return partition, second_eig
def hierarchical_spectral_clustering(values, k, sigma):
    weight_matrix = create_graph(values, k, sigma)
    linkage_matrix = linkage(values, method='ward')
    return linkage_matrix, weight_matrix
def node_distance(linkage_matrix, point1, point2, num_points):
    node1, node2 = point1, point2
    if node1 < num_points and node2 < num_points:
        return 1
    while node1 >= num_points or node2 >= num_points:
        if node1 >= num_points:
            node1 = int(linkage_matrix[node1 - num_points, 0])
        elif node2 >= num_points:
            node2 = int(linkage_matrix[node2 - num_points, 0])
        else:
            if linkage_matrix[node1 - num_points, 0] == linkage_matrix[node2 - num_points, 0]:
                break
            if linkage_matrix[node1 - num_points, 2] < linkage_matrix[node2 - num_points, 2]:
                node1 = int(linkage_matrix[node1 - num_points, 0])
            else:
                node2 = int(linkage_matrix[node2 - num_points, 0])
    return linkage_matrix[node1 - num_points, 2]
def dasgupta_cost(linkage_matrix, weight_matrix):
    num_points = weight_matrix.shape[0]
    total_cost = 0
    for point1 in range(num_points):
        for point2 in range(point1 + 1, num_points):
            if weight_matrix[point1, point2] > 0:
                lca_dist = node_distance(linkage_matrix, point1, point2, num_points)
                total_cost += weight_matrix[point1, point2] * lca_dist
    return total_cost
k =10
linkage_matrix, weight_matrix = hierarchical_spectral_clustering(values, k, sigma)
cost = dasgupta_cost(linkage_matrix, weight_matrix)
print("dasgupta_cost for k=10",cost)

dasgupta_cost for k=10 1562.0117040759308


In [ ]:
##### جدول مقایسه

In [116]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import AgglomerativeClustering
from scipy.spatial.distance import pdist
from scipy.linalg import eigh
from scipy.cluster.hierarchy import linkage
link = ["https://cs.joensuu.fi/sipu/datasets/pathbased.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
values = dataset[["x1", "x2"]].values
distances = pdist(values)
sigma = np.mean(distances)
def gaussian_kernel(x, y, sigma):
    return np.exp(-np.linalg.norm(x - y)**2 / (2 * sigma**2))
def create_spectral_graph(values, k, sigma):
    num_points = len(values)
    knn = NearestNeighbors(n_neighbors=k)
    knn.fit(values)
    distances, indices = knn.kneighbors(values)
    weight_matrix = np.zeros((num_points, num_points))
    for i in range(num_points):
        for j, idx in zip(range(k), indices[i]):
            if i != idx:
                weight_matrix[i, idx] = gaussian_kernel(values[i], values[idx], sigma)
                weight_matrix[idx, i] = weight_matrix[i, idx]
    return weight_matrix
def spectral_laplacian_matrix(weight_matrix):
    degree_matrix = np.diag(np.sum(weight_matrix, axis=1))
    laplacian = degree_matrix - weight_matrix
    return laplacian
def spectral_second_eigenvector(laplacian):
    eigenvalues, eigenvectors = eigh(laplacian)
    return eigenvectors[:, 1]
def spectral_dasgupta_cost(linkage_matrix, weight_matrix):
    num_points = weight_matrix.shape[0]
    total_cost = 0
    for point1 in range(num_points):
        for point2 in range(point1 + 1, num_points):
            if weight_matrix[point1, point2] > 0:
                lca_dist = node_distance(linkage_matrix, point1, point2, num_points)
                total_cost += weight_matrix[point1, point2] * lca_dist
    return total_cost
def create_agglomerative_graph(values, sigma):
    num_points = len(values)
    weight_matrix = np.zeros((num_points, num_points))
    for i in range(num_points):
        for j in range(i + 1, num_points):
            weight_matrix[i, j] = gaussian_kernel(values[i], values[j], sigma)
            weight_matrix[j, i] = weight_matrix[i, j]
    return weight_matrix
agglomerative_clustering = AgglomerativeClustering(n_clusters=None, distance_threshold=1.5, linkage='average')
agglomerative_clustering.fit(values)
dataset["new_labels"] = agglomerative_clustering.labels_
linkage_matrix_agglomerative = agglomerative_clustering.children_
weight_matrix_agglomerative = create_agglomerative_graph(values, sigma)
cost_agglomerative = spectral_dasgupta_cost(linkage_matrix_agglomerative, weight_matrix_agglomerative)
k_values = [3, 5, 10]
cost_spectral_values = []
for k in k_values:
    weight_matrix_spectral = create_spectral_graph(values, k, sigma)
    laplacian_spectral = spectral_laplacian_matrix(weight_matrix_spectral)
    second_eig_spectral = spectral_second_eigenvector(laplacian_spectral)
    linkage_matrix_spectral = linkage(values, method='ward')
    cost_spectral = spectral_dasgupta_cost(linkage_matrix_spectral, weight_matrix_spectral)
    cost_spectral_values.append(cost_spectral)
data = {"Method": ["Spectral Clustering (k=3)", "Spectral Clustering (k=5)", "Spectral Clustering (k=10)", "Agglomerative Clustering"],
    "Dasgupta Cost": cost_spectral_values + [cost_agglomerative]}
df_result = pd.DataFrame(data)
print(df_result)

                       Method  Dasgupta Cost
0   Spectral Clustering (k=3)     383.885071
1   Spectral Clustering (k=5)     727.455505
2  Spectral Clustering (k=10)    1562.011704
3    Agglomerative Clustering   27067.342382


In [ ]:
############# متن چهارم(spiral.txt)

In [ ]:
#####خوشه بندی agglomorative

In [147]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
from sklearn.cluster import AgglomerativeClustering
from scipy.spatial.distance import pdist
link = ["https://cs.joensuu.fi/sipu/datasets/spiral.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
values = dataset[["x1", "x2"]].values
distances = pdist(values)
sigma = np.mean(distances)
def gaussian_kernel(x, y, sigma):
    return np.exp(-np.linalg.norm(x - y)**2 / (2 * sigma**2))
num_points = len(values)
weight_matrix = np.zeros((num_points, num_points))
for i in range(num_points):
    for j in range(i + 1, num_points):
        weight_matrix[i, j] = gaussian_kernel(values[i], values[j], sigma)
        weight_matrix[j, i] = weight_matrix[i, j]
agglomerative_clustering = AgglomerativeClustering(n_clusters=None, distance_threshold=1.5, linkage='average')
agglomerative_clustering.fit(values)
def node_distance(linkage_matrix, point1, point2, num_points):
    node1, node2 = point1, point2
    if node1 < num_points and node2 < num_points:
        return 1
    while node1 >= num_points or node2 >= num_points:
        if node1 >= num_points:
            node1 = int(linkage_matrix[node1 - num_points, 0])
        elif node2 >= num_points:
            node2 = int(linkage_matrix[node2 - num_points, 0])
        else:
            if linkage_matrix[node1 - num_points, 0] == linkage_matrix[node2 - num_points, 0]:
                break
            if linkage_matrix[node1 - num_points, 2] < linkage_matrix[node2 - num_points, 2]:
                node1 = int(linkage_matrix[node1 - num_points, 0])
            else:
                node2 = int(linkage_matrix[node2 - num_points, 0])
    return linkage_matrix[node1 - num_points, 2]
def dasgupta_cost(linkage_matrix, weight_matrix):
    num_points = weight_matrix.shape[0]
    total_cost = 0
    for point1 in range(num_points):
        for point2 in range(point1 + 1, num_points):
            if weight_matrix[point1, point2] > 0:
                lca_dist = node_distance(linkage_matrix, point1, point2, num_points)
                total_cost += weight_matrix[point1, point2] * lca_dist
    return total_cost
linkage_matrix = agglomerative_clustering.children_
weight_matrix = weight_matrix
cost = dasgupta_cost(linkage_matrix, weight_matrix)
print("cost_dasgupta", cost)

cost_dasgupta 29415.44910564478


In [ ]:
#### خوشه بندی سلسله مراتبی طیفی برای k=3

In [122]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import pdist
from scipy.linalg import eigh
from scipy.cluster.hierarchy import linkage, fcluster
link= ["https://cs.joensuu.fi/sipu/datasets/spiral.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
values = dataset[["x1", "x2"]].values
distances = pdist(values)
sigma = np.mean(distances)
distances = pdist(values)
sigma = np.mean(distances)
def gaussian_kernel(x, y, sigma):
    return np.exp(-np.linalg.norm(x - y)**2 / (2 * sigma**2))
def create_graph(values, k, sigma):
    num_points = len(values)
    knn = NearestNeighbors(n_neighbors=k)
    knn.fit(values)
    distances, indices = knn.kneighbors(values)
    weight_matrix = np.zeros((num_points, num_points))
    for i in range(num_points):
        for j, idx in zip(range(k), indices[i]):
            if i != idx:
                weight_matrix[i, idx] = gaussian_kernel(values[i], values[idx], sigma)
                weight_matrix[idx, i] = weight_matrix[i, idx]
    return weight_matrix
def laplacian_matrix(weight_matrix):
    degree_matrix = np.diag(np.sum(weight_matrix, axis=1))
    laplacian = degree_matrix - weight_matrix
    return laplacian
def second_eigenvector(laplacian):
    eigenvalues, eigenvectors = eigh(laplacian)
    return eigenvectors[:, 1]
def spectral_cut(values, weight_matrix, k, sigma):
    W = create_graph(values, k, sigma)
    L = laplacian_matrix(W)
    second_eig = second_eigenvector(L)
    partition = np.sign(second_eig)
    return partition, second_eig
def hierarchical_spectral_clustering(values, k, sigma):
    weight_matrix = create_graph(values, k, sigma)
    linkage_matrix = linkage(values, method='ward')
    return linkage_matrix, weight_matrix
def node_distance(linkage_matrix, point1, point2, num_points):
    node1, node2 = point1, point2
    if node1 < num_points and node2 < num_points:
        return 1
    while node1 >= num_points or node2 >= num_points:
        if node1 >= num_points:
            node1 = int(linkage_matrix[node1 - num_points, 0])
        elif node2 >= num_points:
            node2 = int(linkage_matrix[node2 - num_points, 0])
        else:
            if linkage_matrix[node1 - num_points, 0] == linkage_matrix[node2 - num_points, 0]:
                break
            if linkage_matrix[node1 - num_points, 2] < linkage_matrix[node2 - num_points, 2]:
                node1 = int(linkage_matrix[node1 - num_points, 0])
            else:
                node2 = int(linkage_matrix[node2 - num_points, 0])
    return linkage_matrix[node1 - num_points, 2]
def dasgupta_cost(linkage_matrix, weight_matrix):
    num_points = weight_matrix.shape[0]
    total_cost = 0
    for point1 in range(num_points):
        for point2 in range(point1 + 1, num_points):
            if weight_matrix[point1, point2] > 0:
                lca_dist = node_distance(linkage_matrix, point1, point2, num_points)
                total_cost += weight_matrix[point1, point2] * lca_dist
    return total_cost
k =3
linkage_matrix, weight_matrix = hierarchical_spectral_clustering(values, k, sigma)
cost = dasgupta_cost(linkage_matrix, weight_matrix)
print("dasgupta_cost for k=3",cost)

dasgupta_cost for k=3 315.5805104355261


In [ ]:
#### خوشه بندی سلسله مراتبی طیفی برای k=5

In [123]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import pdist
from scipy.linalg import eigh
from scipy.cluster.hierarchy import linkage, fcluster
link= ["https://cs.joensuu.fi/sipu/datasets/spiral.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
values = dataset[["x1", "x2"]].values
distances = pdist(values)
sigma = np.mean(distances)
distances = pdist(values)
sigma = np.mean(distances)
def gaussian_kernel(x, y, sigma):
    return np.exp(-np.linalg.norm(x - y)**2 / (2 * sigma**2))
def create_graph(values, k, sigma):
    num_points = len(values)
    knn = NearestNeighbors(n_neighbors=k)
    knn.fit(values)
    distances, indices = knn.kneighbors(values)
    weight_matrix = np.zeros((num_points, num_points))
    for i in range(num_points):
        for j, idx in zip(range(k), indices[i]):
            if i != idx:
                weight_matrix[i, idx] = gaussian_kernel(values[i], values[idx], sigma)
                weight_matrix[idx, i] = weight_matrix[i, idx]
    return weight_matrix
def laplacian_matrix(weight_matrix):
    degree_matrix = np.diag(np.sum(weight_matrix, axis=1))
    laplacian = degree_matrix - weight_matrix
    return laplacian
def second_eigenvector(laplacian):
    eigenvalues, eigenvectors = eigh(laplacian)
    return eigenvectors[:, 1]
def spectral_cut(values, weight_matrix, k, sigma):
    W = create_graph(values, k, sigma)
    L = laplacian_matrix(W)
    second_eig = second_eigenvector(L)
    partition = np.sign(second_eig)
    return partition, second_eig
def hierarchical_spectral_clustering(values, k, sigma):
    weight_matrix = create_graph(values, k, sigma)
    linkage_matrix = linkage(values, method='ward')
    return linkage_matrix, weight_matrix
def node_distance(linkage_matrix, point1, point2, num_points):
    node1, node2 = point1, point2
    if node1 < num_points and node2 < num_points:
        return 1
    while node1 >= num_points or node2 >= num_points:
        if node1 >= num_points:
            node1 = int(linkage_matrix[node1 - num_points, 0])
        elif node2 >= num_points:
            node2 = int(linkage_matrix[node2 - num_points, 0])
        else:
            if linkage_matrix[node1 - num_points, 0] == linkage_matrix[node2 - num_points, 0]:
                break
            if linkage_matrix[node1 - num_points, 2] < linkage_matrix[node2 - num_points, 2]:
                node1 = int(linkage_matrix[node1 - num_points, 0])
            else:
                node2 = int(linkage_matrix[node2 - num_points, 0])
    return linkage_matrix[node1 - num_points, 2]
def dasgupta_cost(linkage_matrix, weight_matrix):
    num_points = weight_matrix.shape[0]
    total_cost = 0
    for point1 in range(num_points):
        for point2 in range(point1 + 1, num_points):
            if weight_matrix[point1, point2] > 0:
                lca_dist = node_distance(linkage_matrix, point1, point2, num_points)
                total_cost += weight_matrix[point1, point2] * lca_dist
    return total_cost
k =5
linkage_matrix, weight_matrix = hierarchical_spectral_clustering(values, k, sigma)
cost = dasgupta_cost(linkage_matrix, weight_matrix)
print("dasgupta_cost for k=5",cost)

dasgupta_cost for k=5 634.8311391937355


In [ ]:
#### خوشه بندی سلسله مراتبی طیفی برای k=10

In [125]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import pdist
from scipy.linalg import eigh
from scipy.cluster.hierarchy import linkage, fcluster
link= ["https://cs.joensuu.fi/sipu/datasets/spiral.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
values = dataset[["x1", "x2"]].values
distances = pdist(values)
sigma = np.mean(distances)
distances = pdist(values)
sigma = np.mean(distances)
def gaussian_kernel(x, y, sigma):
    return np.exp(-np.linalg.norm(x - y)**2 / (2 * sigma**2))
def create_graph(values, k, sigma):
    num_points = len(values)
    knn = NearestNeighbors(n_neighbors=k)
    knn.fit(values)
    distances, indices = knn.kneighbors(values)
    weight_matrix = np.zeros((num_points, num_points))
    for i in range(num_points):
        for j, idx in zip(range(k), indices[i]):
            if i != idx:
                weight_matrix[i, idx] = gaussian_kernel(values[i], values[idx], sigma)
                weight_matrix[idx, i] = weight_matrix[i, idx]
    return weight_matrix
def laplacian_matrix(weight_matrix):
    degree_matrix = np.diag(np.sum(weight_matrix, axis=1))
    laplacian = degree_matrix - weight_matrix
    return laplacian
def second_eigenvector(laplacian):
    eigenvalues, eigenvectors = eigh(laplacian)
    return eigenvectors[:, 1]
def spectral_cut(values, weight_matrix, k, sigma):
    W = create_graph(values, k, sigma)
    L = laplacian_matrix(W)
    second_eig = second_eigenvector(L)
    partition = np.sign(second_eig)
    return partition, second_eig
def hierarchical_spectral_clustering(values, k, sigma):
    weight_matrix = create_graph(values, k, sigma)
    linkage_matrix = linkage(values, method='ward')
    return linkage_matrix, weight_matrix
def node_distance(linkage_matrix, point1, point2, num_points):
    node1, node2 = point1, point2
    if node1 < num_points and node2 < num_points:
        return 1
    while node1 >= num_points or node2 >= num_points:
        if node1 >= num_points:
            node1 = int(linkage_matrix[node1 - num_points, 0])
        elif node2 >= num_points:
            node2 = int(linkage_matrix[node2 - num_points, 0])
        else:
            if linkage_matrix[node1 - num_points, 0] == linkage_matrix[node2 - num_points, 0]:
                break
            if linkage_matrix[node1 - num_points, 2] < linkage_matrix[node2 - num_points, 2]:
                node1 = int(linkage_matrix[node1 - num_points, 0])
            else:
                node2 = int(linkage_matrix[node2 - num_points, 0])
    return linkage_matrix[node1 - num_points, 2]
def dasgupta_cost(linkage_matrix, weight_matrix):
    num_points = weight_matrix.shape[0]
    total_cost = 0
    for point1 in range(num_points):
        for point2 in range(point1 + 1, num_points):
            if weight_matrix[point1, point2] > 0:
                lca_dist = node_distance(linkage_matrix, point1, point2, num_points)
                total_cost += weight_matrix[point1, point2] * lca_dist
    return total_cost
k =10
linkage_matrix, weight_matrix = hierarchical_spectral_clustering(values, k, sigma)
cost = dasgupta_cost(linkage_matrix, weight_matrix)
print("dasgupta_cost for k=10",cost)

dasgupta_cost for k=10 1544.4077652542774


In [ ]:
##### جدول مقایسه

In [126]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import AgglomerativeClustering
from scipy.spatial.distance import pdist
from scipy.linalg import eigh
from scipy.cluster.hierarchy import linkage
link = ["https://cs.joensuu.fi/sipu/datasets/spiral.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
values = dataset[["x1", "x2"]].values
distances = pdist(values)
sigma = np.mean(distances)
def gaussian_kernel(x, y, sigma):
    return np.exp(-np.linalg.norm(x - y)**2 / (2 * sigma**2))
def create_spectral_graph(values, k, sigma):
    num_points = len(values)
    knn = NearestNeighbors(n_neighbors=k)
    knn.fit(values)
    distances, indices = knn.kneighbors(values)
    weight_matrix = np.zeros((num_points, num_points))
    for i in range(num_points):
        for j, idx in zip(range(k), indices[i]):
            if i != idx:
                weight_matrix[i, idx] = gaussian_kernel(values[i], values[idx], sigma)
                weight_matrix[idx, i] = weight_matrix[i, idx]
    return weight_matrix
def spectral_laplacian_matrix(weight_matrix):
    degree_matrix = np.diag(np.sum(weight_matrix, axis=1))
    laplacian = degree_matrix - weight_matrix
    return laplacian
def spectral_second_eigenvector(laplacian):
    eigenvalues, eigenvectors = eigh(laplacian)
    return eigenvectors[:, 1]
def spectral_dasgupta_cost(linkage_matrix, weight_matrix):
    num_points = weight_matrix.shape[0]
    total_cost = 0
    for point1 in range(num_points):
        for point2 in range(point1 + 1, num_points):
            if weight_matrix[point1, point2] > 0:
                lca_dist = node_distance(linkage_matrix, point1, point2, num_points)
                total_cost += weight_matrix[point1, point2] * lca_dist
    return total_cost
def create_agglomerative_graph(values, sigma):
    num_points = len(values)
    weight_matrix = np.zeros((num_points, num_points))
    for i in range(num_points):
        for j in range(i + 1, num_points):
            weight_matrix[i, j] = gaussian_kernel(values[i], values[j], sigma)
            weight_matrix[j, i] = weight_matrix[i, j]
    return weight_matrix
agglomerative_clustering = AgglomerativeClustering(n_clusters=None, distance_threshold=1.5, linkage='average')
agglomerative_clustering.fit(values)
dataset["new_labels"] = agglomerative_clustering.labels_
linkage_matrix_agglomerative = agglomerative_clustering.children_
weight_matrix_agglomerative = create_agglomerative_graph(values, sigma)
cost_agglomerative = spectral_dasgupta_cost(linkage_matrix_agglomerative, weight_matrix_agglomerative)
k_values = [3, 5, 10]
cost_spectral_values = []
for k in k_values:
    weight_matrix_spectral = create_spectral_graph(values, k, sigma)
    laplacian_spectral = spectral_laplacian_matrix(weight_matrix_spectral)
    second_eig_spectral = spectral_second_eigenvector(laplacian_spectral)
    linkage_matrix_spectral = linkage(values, method='ward')
    cost_spectral = spectral_dasgupta_cost(linkage_matrix_spectral, weight_matrix_spectral)
    cost_spectral_values.append(cost_spectral)
data = {"Method": ["Spectral Clustering (k=3)", "Spectral Clustering (k=5)", "Spectral Clustering (k=10)", "Agglomerative Clustering"],
    "Dasgupta Cost": cost_spectral_values + [cost_agglomerative]}
df_result = pd.DataFrame(data)
print(df_result)

                       Method  Dasgupta Cost
0   Spectral Clustering (k=3)     315.580510
1   Spectral Clustering (k=5)     634.831139
2  Spectral Clustering (k=10)    1544.407765
3    Agglomerative Clustering   29415.449106


In [ ]:
########## متن پنجم(D31.txt)

In [148]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
from sklearn.cluster import AgglomerativeClustering
from scipy.spatial.distance import pdist
link = ["https://cs.joensuu.fi/sipu/datasets/D31.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
values = dataset[["x1", "x2"]].values
distances = pdist(values)
sigma = np.mean(distances)
def gaussian_kernel(x, y, sigma):
    return np.exp(-np.linalg.norm(x - y)**2 / (2 * sigma**2))
num_points = len(values)
weight_matrix = np.zeros((num_points, num_points))
for i in range(num_points):
    for j in range(i + 1, num_points):
        weight_matrix[i, j] = gaussian_kernel(values[i], values[j], sigma)
        weight_matrix[j, i] = weight_matrix[i, j]
agglomerative_clustering = AgglomerativeClustering(n_clusters=None, distance_threshold=1.5, linkage='average')
agglomerative_clustering.fit(values)
def node_distance(linkage_matrix, point1, point2, num_points):
    node1, node2 = point1, point2
    if node1 < num_points and node2 < num_points:
        return 1
    while node1 >= num_points or node2 >= num_points:
        if node1 >= num_points:
            node1 = int(linkage_matrix[node1 - num_points, 0])
        elif node2 >= num_points:
            node2 = int(linkage_matrix[node2 - num_points, 0])
        else:
            if linkage_matrix[node1 - num_points, 0] == linkage_matrix[node2 - num_points, 0]:
                break
            if linkage_matrix[node1 - num_points, 2] < linkage_matrix[node2 - num_points, 2]:
                node1 = int(linkage_matrix[node1 - num_points, 0])
            else:
                node2 = int(linkage_matrix[node2 - num_points, 0])
    return linkage_matrix[node1 - num_points, 2]
def dasgupta_cost(linkage_matrix, weight_matrix):
    num_points = weight_matrix.shape[0]
    total_cost = 0
    for point1 in range(num_points):
        for point2 in range(point1 + 1, num_points):
            if weight_matrix[point1, point2] > 0:
                lca_dist = node_distance(linkage_matrix, point1, point2, num_points)
                total_cost += weight_matrix[point1, point2] * lca_dist
    return total_cost
linkage_matrix = agglomerative_clustering.children_
weight_matrix = weight_matrix
cost = dasgupta_cost(linkage_matrix, weight_matrix)
print("cost_dasgupta", cost)

cost_dasgupta 2898836.994617578


In [ ]:
#### خوشه بندی سلسله مراتبی طیفی برای k=3

In [98]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import pdist
from scipy.linalg import eigh
from scipy.cluster.hierarchy import linkage, fcluster
link= ["https://cs.joensuu.fi/sipu/datasets/D31.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
values = dataset[["x1", "x2"]].values
distances = pdist(values)
sigma = np.mean(distances)
distances = pdist(values)
sigma = np.mean(distances)
def gaussian_kernel(x, y, sigma):
    return np.exp(-np.linalg.norm(x - y)**2 / (2 * sigma**2))
def create_graph(values, k, sigma):
    num_points = len(values)
    knn = NearestNeighbors(n_neighbors=k)
    knn.fit(values)
    distances, indices = knn.kneighbors(values)
    weight_matrix = np.zeros((num_points, num_points))
    for i in range(num_points):
        for j, idx in zip(range(k), indices[i]):
            if i != idx:
                weight_matrix[i, idx] = gaussian_kernel(values[i], values[idx], sigma)
                weight_matrix[idx, i] = weight_matrix[i, idx]
    return weight_matrix
def laplacian_matrix(weight_matrix):
    degree_matrix = np.diag(np.sum(weight_matrix, axis=1))
    laplacian = degree_matrix - weight_matrix
    return laplacian
def second_eigenvector(laplacian):
    eigenvalues, eigenvectors = eigh(laplacian)
    return eigenvectors[:, 1]
def spectral_cut(values, weight_matrix, k, sigma):
    W = create_graph(values, k, sigma)
    L = laplacian_matrix(W)
    second_eig = second_eigenvector(L)
    partition = np.sign(second_eig)
    return partition, second_eig
def hierarchical_spectral_clustering(values, k, sigma):
    weight_matrix = create_graph(values, k, sigma)
    linkage_matrix = linkage(values, method='ward')
    return linkage_matrix, weight_matrix
def node_distance(linkage_matrix, point1, point2, num_points):
    node1, node2 = point1, point2
    if node1 < num_points and node2 < num_points:
        return 1
    while node1 >= num_points or node2 >= num_points:
        if node1 >= num_points:
            node1 = int(linkage_matrix[node1 - num_points, 0])
        elif node2 >= num_points:
            node2 = int(linkage_matrix[node2 - num_points, 0])
        else:
            if linkage_matrix[node1 - num_points, 0] == linkage_matrix[node2 - num_points, 0]:
                break
            if linkage_matrix[node1 - num_points, 2] < linkage_matrix[node2 - num_points, 2]:
                node1 = int(linkage_matrix[node1 - num_points, 0])
            else:
                node2 = int(linkage_matrix[node2 - num_points, 0])
    return linkage_matrix[node1 - num_points, 2]
def dasgupta_cost(linkage_matrix, weight_matrix):
    num_points = weight_matrix.shape[0]
    total_cost = 0
    for point1 in range(num_points):
        for point2 in range(point1 + 1, num_points):
            if weight_matrix[point1, point2] > 0:
                lca_dist = node_distance(linkage_matrix, point1, point2, num_points)
                total_cost += weight_matrix[point1, point2] * lca_dist
    return total_cost
k =3
linkage_matrix, weight_matrix = hierarchical_spectral_clustering(values, k, sigma)
cost = dasgupta_cost(linkage_matrix, weight_matrix)
print("dasgupta_cost for k=3",cost)

dasgupta_cost for k=3 4109.962342313901


In [ ]:
#### خوشه بندی سلسله مراتبی طیفی برای k=5

In [99]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import pdist
from scipy.linalg import eigh
from scipy.cluster.hierarchy import linkage, fcluster
link= ["https://cs.joensuu.fi/sipu/datasets/D31.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
values = dataset[["x1", "x2"]].values
distances = pdist(values)
sigma = np.mean(distances)
distances = pdist(values)
sigma = np.mean(distances)
def gaussian_kernel(x, y, sigma):
    return np.exp(-np.linalg.norm(x - y)**2 / (2 * sigma**2))
def create_graph(values, k, sigma):
    num_points = len(values)
    knn = NearestNeighbors(n_neighbors=k)
    knn.fit(values)
    distances, indices = knn.kneighbors(values)
    weight_matrix = np.zeros((num_points, num_points))
    for i in range(num_points):
        for j, idx in zip(range(k), indices[i]):
            if i != idx:
                weight_matrix[i, idx] = gaussian_kernel(values[i], values[idx], sigma)
                weight_matrix[idx, i] = weight_matrix[i, idx]
    return weight_matrix
def laplacian_matrix(weight_matrix):
    degree_matrix = np.diag(np.sum(weight_matrix, axis=1))
    laplacian = degree_matrix - weight_matrix
    return laplacian
def second_eigenvector(laplacian):
    eigenvalues, eigenvectors = eigh(laplacian)
    return eigenvectors[:, 1]
def spectral_cut(values, weight_matrix, k, sigma):
    W = create_graph(values, k, sigma)
    L = laplacian_matrix(W)
    second_eig = second_eigenvector(L)
    partition = np.sign(second_eig)
    return partition, second_eig
def hierarchical_spectral_clustering(values, k, sigma):
    weight_matrix = create_graph(values, k, sigma)
    linkage_matrix = linkage(values, method='ward')
    return linkage_matrix, weight_matrix
def node_distance(linkage_matrix, point1, point2, num_points):
    node1, node2 = point1, point2
    if node1 < num_points and node2 < num_points:
        return 1
    while node1 >= num_points or node2 >= num_points:
        if node1 >= num_points:
            node1 = int(linkage_matrix[node1 - num_points, 0])
        elif node2 >= num_points:
            node2 = int(linkage_matrix[node2 - num_points, 0])
        else:
            if linkage_matrix[node1 - num_points, 0] == linkage_matrix[node2 - num_points, 0]:
                break
            if linkage_matrix[node1 - num_points, 2] < linkage_matrix[node2 - num_points, 2]:
                node1 = int(linkage_matrix[node1 - num_points, 0])
            else:
                node2 = int(linkage_matrix[node2 - num_points, 0])
    return linkage_matrix[node1 - num_points, 2]
def dasgupta_cost(linkage_matrix, weight_matrix):
    num_points = weight_matrix.shape[0]
    total_cost = 0
    for point1 in range(num_points):
        for point2 in range(point1 + 1, num_points):
            if weight_matrix[point1, point2] > 0:
                lca_dist = node_distance(linkage_matrix, point1, point2, num_points)
                total_cost += weight_matrix[point1, point2] * lca_dist
    return total_cost
k =5
linkage_matrix, weight_matrix = hierarchical_spectral_clustering(values, k, sigma)
cost = dasgupta_cost(linkage_matrix, weight_matrix)
print("dasgupta_cost for k=5",cost)

dasgupta_cost for k=5 7855.864800217752


In [ ]:
#### خوشه بندی سلسله مراتبی طیفی برای k=10

In [101]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import pdist
from scipy.linalg import eigh
from scipy.cluster.hierarchy import linkage, fcluster
link= ["https://cs.joensuu.fi/sipu/datasets/D31.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
values = dataset[["x1", "x2"]].values
distances = pdist(values)
sigma = np.mean(distances)
distances = pdist(values)
sigma = np.mean(distances)
def gaussian_kernel(x, y, sigma):
    return np.exp(-np.linalg.norm(x - y)**2 / (2 * sigma**2))
def create_graph(values, k, sigma):
    num_points = len(values)
    knn = NearestNeighbors(n_neighbors=k)
    knn.fit(values)
    distances, indices = knn.kneighbors(values)
    weight_matrix = np.zeros((num_points, num_points))
    for i in range(num_points):
        for j, idx in zip(range(k), indices[i]):
            if i != idx:
                weight_matrix[i, idx] = gaussian_kernel(values[i], values[idx], sigma)
                weight_matrix[idx, i] = weight_matrix[i, idx]
    return weight_matrix
def laplacian_matrix(weight_matrix):
    degree_matrix = np.diag(np.sum(weight_matrix, axis=1))
    laplacian = degree_matrix - weight_matrix
    return laplacian
def second_eigenvector(laplacian):
    eigenvalues, eigenvectors = eigh(laplacian)
    return eigenvectors[:, 1]
def spectral_cut(values, weight_matrix, k, sigma):
    W = create_graph(values, k, sigma)
    L = laplacian_matrix(W)
    second_eig = second_eigenvector(L)
    partition = np.sign(second_eig)
    return partition, second_eig
def hierarchical_spectral_clustering(values, k, sigma):
    weight_matrix = create_graph(values, k, sigma)
    linkage_matrix = linkage(values, method='ward')
    return linkage_matrix, weight_matrix
def node_distance(linkage_matrix, point1, point2, num_points):
    node1, node2 = point1, point2
    if node1 < num_points and node2 < num_points:
        return 1
    while node1 >= num_points or node2 >= num_points:
        if node1 >= num_points:
            node1 = int(linkage_matrix[node1 - num_points, 0])
        elif node2 >= num_points:
            node2 = int(linkage_matrix[node2 - num_points, 0])
        else:
            if linkage_matrix[node1 - num_points, 0] == linkage_matrix[node2 - num_points, 0]:
                break
            if linkage_matrix[node1 - num_points, 2] < linkage_matrix[node2 - num_points, 2]:
                node1 = int(linkage_matrix[node1 - num_points, 0])
            else:
                node2 = int(linkage_matrix[node2 - num_points, 0])
    return linkage_matrix[node1 - num_points, 2]
def dasgupta_cost(linkage_matrix, weight_matrix):
    num_points = weight_matrix.shape[0]
    total_cost = 0
    for point1 in range(num_points):
        for point2 in range(point1 + 1, num_points):
            if weight_matrix[point1, point2] > 0:
                lca_dist = node_distance(linkage_matrix, point1, point2, num_points)
                total_cost += weight_matrix[point1, point2] * lca_dist
    return total_cost
k =10
linkage_matrix, weight_matrix = hierarchical_spectral_clustering(values, k, sigma)
cost = dasgupta_cost(linkage_matrix, weight_matrix)
print("dasgupta_cost for k=10",cost)

dasgupta_cost for k=10 17408.30316642998


In [ ]:
##### جدول مقایسه

In [114]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import AgglomerativeClustering
from scipy.spatial.distance import pdist
from scipy.linalg import eigh
from scipy.cluster.hierarchy import linkage
link = ["https://cs.joensuu.fi/sipu/datasets/D31.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
values = dataset[["x1", "x2"]].values
distances = pdist(values)
sigma = np.mean(distances)
def gaussian_kernel(x, y, sigma):
    return np.exp(-np.linalg.norm(x - y)**2 / (2 * sigma**2))
def create_spectral_graph(values, k, sigma):
    num_points = len(values)
    knn = NearestNeighbors(n_neighbors=k)
    knn.fit(values)
    distances, indices = knn.kneighbors(values)
    weight_matrix = np.zeros((num_points, num_points))
    for i in range(num_points):
        for j, idx in zip(range(k), indices[i]):
            if i != idx:
                weight_matrix[i, idx] = gaussian_kernel(values[i], values[idx], sigma)
                weight_matrix[idx, i] = weight_matrix[i, idx]
    return weight_matrix
def spectral_laplacian_matrix(weight_matrix):
    degree_matrix = np.diag(np.sum(weight_matrix, axis=1))
    laplacian = degree_matrix - weight_matrix
    return laplacian
def spectral_second_eigenvector(laplacian):
    eigenvalues, eigenvectors = eigh(laplacian)
    return eigenvectors[:, 1]
def spectral_dasgupta_cost(linkage_matrix, weight_matrix):
    num_points = weight_matrix.shape[0]
    total_cost = 0
    for point1 in range(num_points):
        for point2 in range(point1 + 1, num_points):
            if weight_matrix[point1, point2] > 0:
                lca_dist = node_distance(linkage_matrix, point1, point2, num_points)
                total_cost += weight_matrix[point1, point2] * lca_dist
    return total_cost
def create_agglomerative_graph(values, sigma):
    num_points = len(values)
    weight_matrix = np.zeros((num_points, num_points))
    for i in range(num_points):
        for j in range(i + 1, num_points):
            weight_matrix[i, j] = gaussian_kernel(values[i], values[j], sigma)
            weight_matrix[j, i] = weight_matrix[i, j]
    return weight_matrix
agglomerative_clustering = AgglomerativeClustering(n_clusters=None, distance_threshold=1.5, linkage='average')
agglomerative_clustering.fit(values)
dataset["new_labels"] = agglomerative_clustering.labels_
linkage_matrix_agglomerative = agglomerative_clustering.children_
weight_matrix_agglomerative = create_agglomerative_graph(values, sigma)
cost_agglomerative = spectral_dasgupta_cost(linkage_matrix_agglomerative, weight_matrix_agglomerative)
k_values = [3, 5, 10]
cost_spectral_values = []
for k in k_values:
    weight_matrix_spectral = create_spectral_graph(values, k, sigma)
    laplacian_spectral = spectral_laplacian_matrix(weight_matrix_spectral)
    second_eig_spectral = spectral_second_eigenvector(laplacian_spectral)
    linkage_matrix_spectral = linkage(values, method='ward')
    cost_spectral = spectral_dasgupta_cost(linkage_matrix_spectral, weight_matrix_spectral)
    cost_spectral_values.append(cost_spectral)
data = {"Method": ["Spectral Clustering (k=3)", "Spectral Clustering (k=5)", "Spectral Clustering (k=10)", "Agglomerative Clustering"],
    "Dasgupta Cost": cost_spectral_values + [cost_agglomerative]}
df_result = pd.DataFrame(data)
print(df_result)

                       Method  Dasgupta Cost
0   Spectral Clustering (k=3)   4.109962e+03
1   Spectral Clustering (k=5)   7.855865e+03
2  Spectral Clustering (k=10)   1.740830e+04
3    Agglomerative Clustering   2.898837e+06


In [ ]:
########## متن ششم(R15.txt)

In [ ]:
#####خوشه بندی agglomorative

In [149]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
from sklearn.cluster import AgglomerativeClustering
from scipy.spatial.distance import pdist
link = ["https://cs.joensuu.fi/sipu/datasets/R15.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
values = dataset[["x1", "x2"]].values
distances = pdist(values)
sigma = np.mean(distances)
def gaussian_kernel(x, y, sigma):
    return np.exp(-np.linalg.norm(x - y)**2 / (2 * sigma**2))
num_points = len(values)
weight_matrix = np.zeros((num_points, num_points))
for i in range(num_points):
    for j in range(i + 1, num_points):
        weight_matrix[i, j] = gaussian_kernel(values[i], values[j], sigma)
        weight_matrix[j, i] = weight_matrix[i, j]
agglomerative_clustering = AgglomerativeClustering(n_clusters=None, distance_threshold=1.5, linkage='average')
agglomerative_clustering.fit(values)
def node_distance(linkage_matrix, point1, point2, num_points):
    node1, node2 = point1, point2
    if node1 < num_points and node2 < num_points:
        return 1
    while node1 >= num_points or node2 >= num_points:
        if node1 >= num_points:
            node1 = int(linkage_matrix[node1 - num_points, 0])
        elif node2 >= num_points:
            node2 = int(linkage_matrix[node2 - num_points, 0])
        else:
            if linkage_matrix[node1 - num_points, 0] == linkage_matrix[node2 - num_points, 0]:
                break
            if linkage_matrix[node1 - num_points, 2] < linkage_matrix[node2 - num_points, 2]:
                node1 = int(linkage_matrix[node1 - num_points, 0])
            else:
                node2 = int(linkage_matrix[node2 - num_points, 0])
    return linkage_matrix[node1 - num_points, 2]
def dasgupta_cost(linkage_matrix, weight_matrix):
    num_points = weight_matrix.shape[0]
    total_cost = 0
    for point1 in range(num_points):
        for point2 in range(point1 + 1, num_points):
            if weight_matrix[point1, point2] > 0:
                lca_dist = node_distance(linkage_matrix, point1, point2, num_points)
                total_cost += weight_matrix[point1, point2] * lca_dist
    return total_cost
linkage_matrix = agglomerative_clustering.children_
weight_matrix = weight_matrix
cost = dasgupta_cost(linkage_matrix, weight_matrix)
print("cost_dasgupta", cost)

cost_dasgupta 108681.25618708035


In [ ]:
#### خوشه بندی سلسله مراتبی طیفی برای k=3

In [129]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import pdist
from scipy.linalg import eigh
from scipy.cluster.hierarchy import linkage, fcluster
link= ["https://cs.joensuu.fi/sipu/datasets/R15.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
values = dataset[["x1", "x2"]].values
distances = pdist(values)
sigma = np.mean(distances)
distances = pdist(values)
sigma = np.mean(distances)
def gaussian_kernel(x, y, sigma):
    return np.exp(-np.linalg.norm(x - y)**2 / (2 * sigma**2))
def create_graph(values, k, sigma):
    num_points = len(values)
    knn = NearestNeighbors(n_neighbors=k)
    knn.fit(values)
    distances, indices = knn.kneighbors(values)
    weight_matrix = np.zeros((num_points, num_points))
    for i in range(num_points):
        for j, idx in zip(range(k), indices[i]):
            if i != idx:
                weight_matrix[i, idx] = gaussian_kernel(values[i], values[idx], sigma)
                weight_matrix[idx, i] = weight_matrix[i, idx]
    return weight_matrix
def laplacian_matrix(weight_matrix):
    degree_matrix = np.diag(np.sum(weight_matrix, axis=1))
    laplacian = degree_matrix - weight_matrix
    return laplacian
def second_eigenvector(laplacian):
    eigenvalues, eigenvectors = eigh(laplacian)
    return eigenvectors[:, 1]
def spectral_cut(values, weight_matrix, k, sigma):
    W = create_graph(values, k, sigma)
    L = laplacian_matrix(W)
    second_eig = second_eigenvector(L)
    partition = np.sign(second_eig)
    return partition, second_eig
def hierarchical_spectral_clustering(values, k, sigma):
    weight_matrix = create_graph(values, k, sigma)
    linkage_matrix = linkage(values, method='ward')
    return linkage_matrix, weight_matrix
def node_distance(linkage_matrix, point1, point2, num_points):
    node1, node2 = point1, point2
    if node1 < num_points and node2 < num_points:
        return 1
    while node1 >= num_points or node2 >= num_points:
        if node1 >= num_points:
            node1 = int(linkage_matrix[node1 - num_points, 0])
        elif node2 >= num_points:
            node2 = int(linkage_matrix[node2 - num_points, 0])
        else:
            if linkage_matrix[node1 - num_points, 0] == linkage_matrix[node2 - num_points, 0]:
                break
            if linkage_matrix[node1 - num_points, 2] < linkage_matrix[node2 - num_points, 2]:
                node1 = int(linkage_matrix[node1 - num_points, 0])
            else:
                node2 = int(linkage_matrix[node2 - num_points, 0])
    return linkage_matrix[node1 - num_points, 2]
def dasgupta_cost(linkage_matrix, weight_matrix):
    num_points = weight_matrix.shape[0]
    total_cost = 0
    for point1 in range(num_points):
        for point2 in range(point1 + 1, num_points):
            if weight_matrix[point1, point2] > 0:
                lca_dist = node_distance(linkage_matrix, point1, point2, num_points)
                total_cost += weight_matrix[point1, point2] * lca_dist
    return total_cost
k =3
linkage_matrix, weight_matrix = hierarchical_spectral_clustering(values, k, sigma)
cost = dasgupta_cost(linkage_matrix, weight_matrix)
print("dasgupta_cost for k=3",cost)

dasgupta_cost for k=3 807.6099809488633


In [ ]:
#### خوشه بندی سلسله مراتبی طیفی برای k=5

In [130]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import pdist
from scipy.linalg import eigh
from scipy.cluster.hierarchy import linkage, fcluster
link= ["https://cs.joensuu.fi/sipu/datasets/R15.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
values = dataset[["x1", "x2"]].values
distances = pdist(values)
sigma = np.mean(distances)
distances = pdist(values)
sigma = np.mean(distances)
def gaussian_kernel(x, y, sigma):
    return np.exp(-np.linalg.norm(x - y)**2 / (2 * sigma**2))
def create_graph(values, k, sigma):
    num_points = len(values)
    knn = NearestNeighbors(n_neighbors=k)
    knn.fit(values)
    distances, indices = knn.kneighbors(values)
    weight_matrix = np.zeros((num_points, num_points))
    for i in range(num_points):
        for j, idx in zip(range(k), indices[i]):
            if i != idx:
                weight_matrix[i, idx] = gaussian_kernel(values[i], values[idx], sigma)
                weight_matrix[idx, i] = weight_matrix[i, idx]
    return weight_matrix
def laplacian_matrix(weight_matrix):
    degree_matrix = np.diag(np.sum(weight_matrix, axis=1))
    laplacian = degree_matrix - weight_matrix
    return laplacian
def second_eigenvector(laplacian):
    eigenvalues, eigenvectors = eigh(laplacian)
    return eigenvectors[:, 1]
def spectral_cut(values, weight_matrix, k, sigma):
    W = create_graph(values, k, sigma)
    L = laplacian_matrix(W)
    second_eig = second_eigenvector(L)
    partition = np.sign(second_eig)
    return partition, second_eig
def hierarchical_spectral_clustering(values, k, sigma):
    weight_matrix = create_graph(values, k, sigma)
    linkage_matrix = linkage(values, method='ward')
    return linkage_matrix, weight_matrix
def node_distance(linkage_matrix, point1, point2, num_points):
    node1, node2 = point1, point2
    if node1 < num_points and node2 < num_points:
        return 1
    while node1 >= num_points or node2 >= num_points:
        if node1 >= num_points:
            node1 = int(linkage_matrix[node1 - num_points, 0])
        elif node2 >= num_points:
            node2 = int(linkage_matrix[node2 - num_points, 0])
        else:
            if linkage_matrix[node1 - num_points, 0] == linkage_matrix[node2 - num_points, 0]:
                break
            if linkage_matrix[node1 - num_points, 2] < linkage_matrix[node2 - num_points, 2]:
                node1 = int(linkage_matrix[node1 - num_points, 0])
            else:
                node2 = int(linkage_matrix[node2 - num_points, 0])
    return linkage_matrix[node1 - num_points, 2]
def dasgupta_cost(linkage_matrix, weight_matrix):
    num_points = weight_matrix.shape[0]
    total_cost = 0
    for point1 in range(num_points):
        for point2 in range(point1 + 1, num_points):
            if weight_matrix[point1, point2] > 0:
                lca_dist = node_distance(linkage_matrix, point1, point2, num_points)
                total_cost += weight_matrix[point1, point2] * lca_dist
    return total_cost
k =5
linkage_matrix, weight_matrix = hierarchical_spectral_clustering(values, k, sigma)
cost = dasgupta_cost(linkage_matrix, weight_matrix)
print("dasgupta_cost for k=5",cost)

dasgupta_cost for k=5 1565.7693710057847


In [ ]:
#### خوشه بندی سلسله مراتبی طیفی برای k=10

In [131]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import pdist
from scipy.linalg import eigh
from scipy.cluster.hierarchy import linkage, fcluster
link= ["https://cs.joensuu.fi/sipu/datasets/R15.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
values = dataset[["x1", "x2"]].values
distances = pdist(values)
sigma = np.mean(distances)
distances = pdist(values)
sigma = np.mean(distances)
def gaussian_kernel(x, y, sigma):
    return np.exp(-np.linalg.norm(x - y)**2 / (2 * sigma**2))
def create_graph(values, k, sigma):
    num_points = len(values)
    knn = NearestNeighbors(n_neighbors=k)
    knn.fit(values)
    distances, indices = knn.kneighbors(values)
    weight_matrix = np.zeros((num_points, num_points))
    for i in range(num_points):
        for j, idx in zip(range(k), indices[i]):
            if i != idx:
                weight_matrix[i, idx] = gaussian_kernel(values[i], values[idx], sigma)
                weight_matrix[idx, i] = weight_matrix[i, idx]
    return weight_matrix
def laplacian_matrix(weight_matrix):
    degree_matrix = np.diag(np.sum(weight_matrix, axis=1))
    laplacian = degree_matrix - weight_matrix
    return laplacian
def second_eigenvector(laplacian):
    eigenvalues, eigenvectors = eigh(laplacian)
    return eigenvectors[:, 1]
def spectral_cut(values, weight_matrix, k, sigma):
    W = create_graph(values, k, sigma)
    L = laplacian_matrix(W)
    second_eig = second_eigenvector(L)
    partition = np.sign(second_eig)
    return partition, second_eig
def hierarchical_spectral_clustering(values, k, sigma):
    weight_matrix = create_graph(values, k, sigma)
    linkage_matrix = linkage(values, method='ward')
    return linkage_matrix, weight_matrix
def node_distance(linkage_matrix, point1, point2, num_points):
    node1, node2 = point1, point2
    if node1 < num_points and node2 < num_points:
        return 1
    while node1 >= num_points or node2 >= num_points:
        if node1 >= num_points:
            node1 = int(linkage_matrix[node1 - num_points, 0])
        elif node2 >= num_points:
            node2 = int(linkage_matrix[node2 - num_points, 0])
        else:
            if linkage_matrix[node1 - num_points, 0] == linkage_matrix[node2 - num_points, 0]:
                break
            if linkage_matrix[node1 - num_points, 2] < linkage_matrix[node2 - num_points, 2]:
                node1 = int(linkage_matrix[node1 - num_points, 0])
            else:
                node2 = int(linkage_matrix[node2 - num_points, 0])
    return linkage_matrix[node1 - num_points, 2]
def dasgupta_cost(linkage_matrix, weight_matrix):
    num_points = weight_matrix.shape[0]
    total_cost = 0
    for point1 in range(num_points):
        for point2 in range(point1 + 1, num_points):
            if weight_matrix[point1, point2] > 0:
                lca_dist = node_distance(linkage_matrix, point1, point2, num_points)
                total_cost += weight_matrix[point1, point2] * lca_dist
    return total_cost
k =10
linkage_matrix, weight_matrix = hierarchical_spectral_clustering(values, k, sigma)
cost = dasgupta_cost(linkage_matrix, weight_matrix)
print("dasgupta_cost for k=10",cost)

dasgupta_cost for k=10 3493.96803105565


In [ ]:
##### جدول مقایسه

In [132]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import AgglomerativeClustering
from scipy.spatial.distance import pdist
from scipy.linalg import eigh
from scipy.cluster.hierarchy import linkage
link = ["https://cs.joensuu.fi/sipu/datasets/R15.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
values = dataset[["x1", "x2"]].values
distances = pdist(values)
sigma = np.mean(distances)
def gaussian_kernel(x, y, sigma):
    return np.exp(-np.linalg.norm(x - y)**2 / (2 * sigma**2))
def create_spectral_graph(values, k, sigma):
    num_points = len(values)
    knn = NearestNeighbors(n_neighbors=k)
    knn.fit(values)
    distances, indices = knn.kneighbors(values)
    weight_matrix = np.zeros((num_points, num_points))
    for i in range(num_points):
        for j, idx in zip(range(k), indices[i]):
            if i != idx:
                weight_matrix[i, idx] = gaussian_kernel(values[i], values[idx], sigma)
                weight_matrix[idx, i] = weight_matrix[i, idx]
    return weight_matrix
def spectral_laplacian_matrix(weight_matrix):
    degree_matrix = np.diag(np.sum(weight_matrix, axis=1))
    laplacian = degree_matrix - weight_matrix
    return laplacian
def spectral_second_eigenvector(laplacian):
    eigenvalues, eigenvectors = eigh(laplacian)
    return eigenvectors[:, 1]
def spectral_dasgupta_cost(linkage_matrix, weight_matrix):
    num_points = weight_matrix.shape[0]
    total_cost = 0
    for point1 in range(num_points):
        for point2 in range(point1 + 1, num_points):
            if weight_matrix[point1, point2] > 0:
                lca_dist = node_distance(linkage_matrix, point1, point2, num_points)
                total_cost += weight_matrix[point1, point2] * lca_dist
    return total_cost
def create_agglomerative_graph(values, sigma):
    num_points = len(values)
    weight_matrix = np.zeros((num_points, num_points))
    for i in range(num_points):
        for j in range(i + 1, num_points):
            weight_matrix[i, j] = gaussian_kernel(values[i], values[j], sigma)
            weight_matrix[j, i] = weight_matrix[i, j]
    return weight_matrix
agglomerative_clustering = AgglomerativeClustering(n_clusters=None, distance_threshold=1.5, linkage='average')
agglomerative_clustering.fit(values)
dataset["new_labels"] = agglomerative_clustering.labels_
linkage_matrix_agglomerative = agglomerative_clustering.children_
weight_matrix_agglomerative = create_agglomerative_graph(values, sigma)
cost_agglomerative = spectral_dasgupta_cost(linkage_matrix_agglomerative, weight_matrix_agglomerative)
k_values = [3, 5, 10]
cost_spectral_values = []
for k in k_values:
    weight_matrix_spectral = create_spectral_graph(values, k, sigma)
    laplacian_spectral = spectral_laplacian_matrix(weight_matrix_spectral)
    second_eig_spectral = spectral_second_eigenvector(laplacian_spectral)
    linkage_matrix_spectral = linkage(values, method='ward')
    cost_spectral = spectral_dasgupta_cost(linkage_matrix_spectral, weight_matrix_spectral)
    cost_spectral_values.append(cost_spectral)
data = {"Method": ["Spectral Clustering (k=3)", "Spectral Clustering (k=5)", "Spectral Clustering (k=10)", "Agglomerative Clustering"],
    "Dasgupta Cost": cost_spectral_values + [cost_agglomerative]}
df_result = pd.DataFrame(data)
print(df_result)

                       Method  Dasgupta Cost
0   Spectral Clustering (k=3)     807.609981
1   Spectral Clustering (k=5)    1565.769371
2  Spectral Clustering (k=10)    3493.968031
3    Agglomerative Clustering  108681.256187


In [ ]:
######## متن هفتم(jain.txt)

In [ ]:
#####خوشه بندی agglomorative

In [150]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
from sklearn.cluster import AgglomerativeClustering
from scipy.spatial.distance import pdist
link = ["https://cs.joensuu.fi/sipu/datasets/jain.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
values = dataset[["x1", "x2"]].values
distances = pdist(values)
sigma = np.mean(distances)
def gaussian_kernel(x, y, sigma):
    return np.exp(-np.linalg.norm(x - y)**2 / (2 * sigma**2))
num_points = len(values)
weight_matrix = np.zeros((num_points, num_points))
for i in range(num_points):
    for j in range(i + 1, num_points):
        weight_matrix[i, j] = gaussian_kernel(values[i], values[j], sigma)
        weight_matrix[j, i] = weight_matrix[i, j]
agglomerative_clustering = AgglomerativeClustering(n_clusters=None, distance_threshold=1.5, linkage='average')
agglomerative_clustering.fit(values)
def node_distance(linkage_matrix, point1, point2, num_points):
    node1, node2 = point1, point2
    if node1 < num_points and node2 < num_points:
        return 1
    while node1 >= num_points or node2 >= num_points:
        if node1 >= num_points:
            node1 = int(linkage_matrix[node1 - num_points, 0])
        elif node2 >= num_points:
            node2 = int(linkage_matrix[node2 - num_points, 0])
        else:
            if linkage_matrix[node1 - num_points, 0] == linkage_matrix[node2 - num_points, 0]:
                break
            if linkage_matrix[node1 - num_points, 2] < linkage_matrix[node2 - num_points, 2]:
                node1 = int(linkage_matrix[node1 - num_points, 0])
            else:
                node2 = int(linkage_matrix[node2 - num_points, 0])
    return linkage_matrix[node1 - num_points, 2]
def dasgupta_cost(linkage_matrix, weight_matrix):
    num_points = weight_matrix.shape[0]
    total_cost = 0
    for point1 in range(num_points):
        for point2 in range(point1 + 1, num_points):
            if weight_matrix[point1, point2] > 0:
                lca_dist = node_distance(linkage_matrix, point1, point2, num_points)
                total_cost += weight_matrix[point1, point2] * lca_dist
    return total_cost
linkage_matrix = agglomerative_clustering.children_
weight_matrix = weight_matrix
cost = dasgupta_cost(linkage_matrix, weight_matrix)
print("cost_dasgupta", cost)

cost_dasgupta 42093.84241554653


In [ ]:
#### خوشه بندی سلسله مراتبی طیفی برای k=3

In [135]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import pdist
from scipy.linalg import eigh
from scipy.cluster.hierarchy import linkage, fcluster
link= ["https://cs.joensuu.fi/sipu/datasets/jain.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
values = dataset[["x1", "x2"]].values
distances = pdist(values)
sigma = np.mean(distances)
distances = pdist(values)
sigma = np.mean(distances)
def gaussian_kernel(x, y, sigma):
    return np.exp(-np.linalg.norm(x - y)**2 / (2 * sigma**2))
def create_graph(values, k, sigma):
    num_points = len(values)
    knn = NearestNeighbors(n_neighbors=k)
    knn.fit(values)
    distances, indices = knn.kneighbors(values)
    weight_matrix = np.zeros((num_points, num_points))
    for i in range(num_points):
        for j, idx in zip(range(k), indices[i]):
            if i != idx:
                weight_matrix[i, idx] = gaussian_kernel(values[i], values[idx], sigma)
                weight_matrix[idx, i] = weight_matrix[i, idx]
    return weight_matrix
def laplacian_matrix(weight_matrix):
    degree_matrix = np.diag(np.sum(weight_matrix, axis=1))
    laplacian = degree_matrix - weight_matrix
    return laplacian
def second_eigenvector(laplacian):
    eigenvalues, eigenvectors = eigh(laplacian)
    return eigenvectors[:, 1]
def spectral_cut(values, weight_matrix, k, sigma):
    W = create_graph(values, k, sigma)
    L = laplacian_matrix(W)
    second_eig = second_eigenvector(L)
    partition = np.sign(second_eig)
    return partition, second_eig
def hierarchical_spectral_clustering(values, k, sigma):
    weight_matrix = create_graph(values, k, sigma)
    linkage_matrix = linkage(values, method='ward')
    return linkage_matrix, weight_matrix
def node_distance(linkage_matrix, point1, point2, num_points):
    node1, node2 = point1, point2
    if node1 < num_points and node2 < num_points:
        return 1
    while node1 >= num_points or node2 >= num_points:
        if node1 >= num_points:
            node1 = int(linkage_matrix[node1 - num_points, 0])
        elif node2 >= num_points:
            node2 = int(linkage_matrix[node2 - num_points, 0])
        else:
            if linkage_matrix[node1 - num_points, 0] == linkage_matrix[node2 - num_points, 0]:
                break
            if linkage_matrix[node1 - num_points, 2] < linkage_matrix[node2 - num_points, 2]:
                node1 = int(linkage_matrix[node1 - num_points, 0])
            else:
                node2 = int(linkage_matrix[node2 - num_points, 0])
    return linkage_matrix[node1 - num_points, 2]
def dasgupta_cost(linkage_matrix, weight_matrix):
    num_points = weight_matrix.shape[0]
    total_cost = 0
    for point1 in range(num_points):
        for point2 in range(point1 + 1, num_points):
            if weight_matrix[point1, point2] > 0:
                lca_dist = node_distance(linkage_matrix, point1, point2, num_points)
                total_cost += weight_matrix[point1, point2] * lca_dist
    return total_cost
k =3
linkage_matrix, weight_matrix = hierarchical_spectral_clustering(values, k, sigma)
cost = dasgupta_cost(linkage_matrix, weight_matrix)
print("dasgupta_cost for k=3",cost)

dasgupta_cost for k=3 478.1763672838005


In [ ]:
#### خوشه بندی سلسله مراتبی طیفی برای k=5

In [136]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import pdist
from scipy.linalg import eigh
from scipy.cluster.hierarchy import linkage, fcluster
link= ["https://cs.joensuu.fi/sipu/datasets/jain.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
values = dataset[["x1", "x2"]].values
distances = pdist(values)
sigma = np.mean(distances)
distances = pdist(values)
sigma = np.mean(distances)
def gaussian_kernel(x, y, sigma):
    return np.exp(-np.linalg.norm(x - y)**2 / (2 * sigma**2))
def create_graph(values, k, sigma):
    num_points = len(values)
    knn = NearestNeighbors(n_neighbors=k)
    knn.fit(values)
    distances, indices = knn.kneighbors(values)
    weight_matrix = np.zeros((num_points, num_points))
    for i in range(num_points):
        for j, idx in zip(range(k), indices[i]):
            if i != idx:
                weight_matrix[i, idx] = gaussian_kernel(values[i], values[idx], sigma)
                weight_matrix[idx, i] = weight_matrix[i, idx]
    return weight_matrix
def laplacian_matrix(weight_matrix):
    degree_matrix = np.diag(np.sum(weight_matrix, axis=1))
    laplacian = degree_matrix - weight_matrix
    return laplacian
def second_eigenvector(laplacian):
    eigenvalues, eigenvectors = eigh(laplacian)
    return eigenvectors[:, 1]
def spectral_cut(values, weight_matrix, k, sigma):
    W = create_graph(values, k, sigma)
    L = laplacian_matrix(W)
    second_eig = second_eigenvector(L)
    partition = np.sign(second_eig)
    return partition, second_eig
def hierarchical_spectral_clustering(values, k, sigma):
    weight_matrix = create_graph(values, k, sigma)
    linkage_matrix = linkage(values, method='ward')
    return linkage_matrix, weight_matrix
def node_distance(linkage_matrix, point1, point2, num_points):
    node1, node2 = point1, point2
    if node1 < num_points and node2 < num_points:
        return 1
    while node1 >= num_points or node2 >= num_points:
        if node1 >= num_points:
            node1 = int(linkage_matrix[node1 - num_points, 0])
        elif node2 >= num_points:
            node2 = int(linkage_matrix[node2 - num_points, 0])
        else:
            if linkage_matrix[node1 - num_points, 0] == linkage_matrix[node2 - num_points, 0]:
                break
            if linkage_matrix[node1 - num_points, 2] < linkage_matrix[node2 - num_points, 2]:
                node1 = int(linkage_matrix[node1 - num_points, 0])
            else:
                node2 = int(linkage_matrix[node2 - num_points, 0])
    return linkage_matrix[node1 - num_points, 2]
def dasgupta_cost(linkage_matrix, weight_matrix):
    num_points = weight_matrix.shape[0]
    total_cost = 0
    for point1 in range(num_points):
        for point2 in range(point1 + 1, num_points):
            if weight_matrix[point1, point2] > 0:
                lca_dist = node_distance(linkage_matrix, point1, point2, num_points)
                total_cost += weight_matrix[point1, point2] * lca_dist
    return total_cost
k =5
linkage_matrix, weight_matrix = hierarchical_spectral_clustering(values, k, sigma)
cost = dasgupta_cost(linkage_matrix, weight_matrix)
print("dasgupta_cost for k=5",cost)

dasgupta_cost for k=5 917.3398473948137


In [ ]:
#### خوشه بندی سلسله مراتبی طیفی برای k=10

In [137]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import pdist
from scipy.linalg import eigh
from scipy.cluster.hierarchy import linkage, fcluster
link= ["https://cs.joensuu.fi/sipu/datasets/jain.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
values = dataset[["x1", "x2"]].values
distances = pdist(values)
sigma = np.mean(distances)
distances = pdist(values)
sigma = np.mean(distances)
def gaussian_kernel(x, y, sigma):
    return np.exp(-np.linalg.norm(x - y)**2 / (2 * sigma**2))
def create_graph(values, k, sigma):
    num_points = len(values)
    knn = NearestNeighbors(n_neighbors=k)
    knn.fit(values)
    distances, indices = knn.kneighbors(values)
    weight_matrix = np.zeros((num_points, num_points))
    for i in range(num_points):
        for j, idx in zip(range(k), indices[i]):
            if i != idx:
                weight_matrix[i, idx] = gaussian_kernel(values[i], values[idx], sigma)
                weight_matrix[idx, i] = weight_matrix[i, idx]
    return weight_matrix
def laplacian_matrix(weight_matrix):
    degree_matrix = np.diag(np.sum(weight_matrix, axis=1))
    laplacian = degree_matrix - weight_matrix
    return laplacian
def second_eigenvector(laplacian):
    eigenvalues, eigenvectors = eigh(laplacian)
    return eigenvectors[:, 1]
def spectral_cut(values, weight_matrix, k, sigma):
    W = create_graph(values, k, sigma)
    L = laplacian_matrix(W)
    second_eig = second_eigenvector(L)
    partition = np.sign(second_eig)
    return partition, second_eig
def hierarchical_spectral_clustering(values, k, sigma):
    weight_matrix = create_graph(values, k, sigma)
    linkage_matrix = linkage(values, method='ward')
    return linkage_matrix, weight_matrix
def node_distance(linkage_matrix, point1, point2, num_points):
    node1, node2 = point1, point2
    if node1 < num_points and node2 < num_points:
        return 1
    while node1 >= num_points or node2 >= num_points:
        if node1 >= num_points:
            node1 = int(linkage_matrix[node1 - num_points, 0])
        elif node2 >= num_points:
            node2 = int(linkage_matrix[node2 - num_points, 0])
        else:
            if linkage_matrix[node1 - num_points, 0] == linkage_matrix[node2 - num_points, 0]:
                break
            if linkage_matrix[node1 - num_points, 2] < linkage_matrix[node2 - num_points, 2]:
                node1 = int(linkage_matrix[node1 - num_points, 0])
            else:
                node2 = int(linkage_matrix[node2 - num_points, 0])
    return linkage_matrix[node1 - num_points, 2]
def dasgupta_cost(linkage_matrix, weight_matrix):
    num_points = weight_matrix.shape[0]
    total_cost = 0
    for point1 in range(num_points):
        for point2 in range(point1 + 1, num_points):
            if weight_matrix[point1, point2] > 0:
                lca_dist = node_distance(linkage_matrix, point1, point2, num_points)
                total_cost += weight_matrix[point1, point2] * lca_dist
    return total_cost
k =10
linkage_matrix, weight_matrix = hierarchical_spectral_clustering(values, k, sigma)
cost = dasgupta_cost(linkage_matrix, weight_matrix)
print("dasgupta_cost for k=10",cost)

dasgupta_cost for k=10 1999.1001318004778


In [ ]:
##### جدول مقایسه

In [138]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import AgglomerativeClustering
from scipy.spatial.distance import pdist
from scipy.linalg import eigh
from scipy.cluster.hierarchy import linkage
link = ["https://cs.joensuu.fi/sipu/datasets/jain.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
values = dataset[["x1", "x2"]].values
distances = pdist(values)
sigma = np.mean(distances)
def gaussian_kernel(x, y, sigma):
    return np.exp(-np.linalg.norm(x - y)**2 / (2 * sigma**2))
def create_spectral_graph(values, k, sigma):
    num_points = len(values)
    knn = NearestNeighbors(n_neighbors=k)
    knn.fit(values)
    distances, indices = knn.kneighbors(values)
    weight_matrix = np.zeros((num_points, num_points))
    for i in range(num_points):
        for j, idx in zip(range(k), indices[i]):
            if i != idx:
                weight_matrix[i, idx] = gaussian_kernel(values[i], values[idx], sigma)
                weight_matrix[idx, i] = weight_matrix[i, idx]
    return weight_matrix
def spectral_laplacian_matrix(weight_matrix):
    degree_matrix = np.diag(np.sum(weight_matrix, axis=1))
    laplacian = degree_matrix - weight_matrix
    return laplacian
def spectral_second_eigenvector(laplacian):
    eigenvalues, eigenvectors = eigh(laplacian)
    return eigenvectors[:, 1]
def spectral_dasgupta_cost(linkage_matrix, weight_matrix):
    num_points = weight_matrix.shape[0]
    total_cost = 0
    for point1 in range(num_points):
        for point2 in range(point1 + 1, num_points):
            if weight_matrix[point1, point2] > 0:
                lca_dist = node_distance(linkage_matrix, point1, point2, num_points)
                total_cost += weight_matrix[point1, point2] * lca_dist
    return total_cost
def create_agglomerative_graph(values, sigma):
    num_points = len(values)
    weight_matrix = np.zeros((num_points, num_points))
    for i in range(num_points):
        for j in range(i + 1, num_points):
            weight_matrix[i, j] = gaussian_kernel(values[i], values[j], sigma)
            weight_matrix[j, i] = weight_matrix[i, j]
    return weight_matrix
agglomerative_clustering = AgglomerativeClustering(n_clusters=None, distance_threshold=1.5, linkage='average')
agglomerative_clustering.fit(values)
dataset["new_labels"] = agglomerative_clustering.labels_
linkage_matrix_agglomerative = agglomerative_clustering.children_
weight_matrix_agglomerative = create_agglomerative_graph(values, sigma)
cost_agglomerative = spectral_dasgupta_cost(linkage_matrix_agglomerative, weight_matrix_agglomerative)
k_values = [3, 5, 10]
cost_spectral_values = []
for k in k_values:
    weight_matrix_spectral = create_spectral_graph(values, k, sigma)
    laplacian_spectral = spectral_laplacian_matrix(weight_matrix_spectral)
    second_eig_spectral = spectral_second_eigenvector(laplacian_spectral)
    linkage_matrix_spectral = linkage(values, method='ward')
    cost_spectral = spectral_dasgupta_cost(linkage_matrix_spectral, weight_matrix_spectral)
    cost_spectral_values.append(cost_spectral)
data = {"Method": ["Spectral Clustering (k=3)", "Spectral Clustering (k=5)", "Spectral Clustering (k=10)", "Agglomerative Clustering"],
    "Dasgupta Cost": cost_spectral_values + [cost_agglomerative]}
df_result = pd.DataFrame(data)
print(df_result)

                       Method  Dasgupta Cost
0   Spectral Clustering (k=3)     478.176367
1   Spectral Clustering (k=5)     917.339847
2  Spectral Clustering (k=10)    1999.100132
3    Agglomerative Clustering   42093.842416


In [ ]:
########## متن هشتم(flame.txt)

In [ ]:
#####خوشه بندی agglomorative

In [151]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
from sklearn.cluster import AgglomerativeClustering
from scipy.spatial.distance import pdist
link = ["https://cs.joensuu.fi/sipu/datasets/flame.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
values = dataset[["x1", "x2"]].values
distances = pdist(values)
sigma = np.mean(distances)
def gaussian_kernel(x, y, sigma):
    return np.exp(-np.linalg.norm(x - y)**2 / (2 * sigma**2))
num_points = len(values)
weight_matrix = np.zeros((num_points, num_points))
for i in range(num_points):
    for j in range(i + 1, num_points):
        weight_matrix[i, j] = gaussian_kernel(values[i], values[j], sigma)
        weight_matrix[j, i] = weight_matrix[i, j]
agglomerative_clustering = AgglomerativeClustering(n_clusters=None, distance_threshold=1.5, linkage='average')
agglomerative_clustering.fit(values)
def node_distance(linkage_matrix, point1, point2, num_points):
    node1, node2 = point1, point2
    if node1 < num_points and node2 < num_points:
        return 1
    while node1 >= num_points or node2 >= num_points:
        if node1 >= num_points:
            node1 = int(linkage_matrix[node1 - num_points, 0])
        elif node2 >= num_points:
            node2 = int(linkage_matrix[node2 - num_points, 0])
        else:
            if linkage_matrix[node1 - num_points, 0] == linkage_matrix[node2 - num_points, 0]:
                break
            if linkage_matrix[node1 - num_points, 2] < linkage_matrix[node2 - num_points, 2]:
                node1 = int(linkage_matrix[node1 - num_points, 0])
            else:
                node2 = int(linkage_matrix[node2 - num_points, 0])
    return linkage_matrix[node1 - num_points, 2]
def dasgupta_cost(linkage_matrix, weight_matrix):
    num_points = weight_matrix.shape[0]
    total_cost = 0
    for point1 in range(num_points):
        for point2 in range(point1 + 1, num_points):
            if weight_matrix[point1, point2] > 0:
                lca_dist = node_distance(linkage_matrix, point1, point2, num_points)
                total_cost += weight_matrix[point1, point2] * lca_dist
    return total_cost
linkage_matrix = agglomerative_clustering.children_
weight_matrix = weight_matrix
cost = dasgupta_cost(linkage_matrix, weight_matrix)
print("cost_dasgupta", cost)

cost_dasgupta 17316.747367472835


In [ ]:
#### خوشه بندی سلسله مراتبی طیفی برای k=3

In [140]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import pdist
from scipy.linalg import eigh
from scipy.cluster.hierarchy import linkage, fcluster
link= ["https://cs.joensuu.fi/sipu/datasets/flame.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
values = dataset[["x1", "x2"]].values
distances = pdist(values)
sigma = np.mean(distances)
distances = pdist(values)
sigma = np.mean(distances)
def gaussian_kernel(x, y, sigma):
    return np.exp(-np.linalg.norm(x - y)**2 / (2 * sigma**2))
def create_graph(values, k, sigma):
    num_points = len(values)
    knn = NearestNeighbors(n_neighbors=k)
    knn.fit(values)
    distances, indices = knn.kneighbors(values)
    weight_matrix = np.zeros((num_points, num_points))
    for i in range(num_points):
        for j, idx in zip(range(k), indices[i]):
            if i != idx:
                weight_matrix[i, idx] = gaussian_kernel(values[i], values[idx], sigma)
                weight_matrix[idx, i] = weight_matrix[i, idx]
    return weight_matrix
def laplacian_matrix(weight_matrix):
    degree_matrix = np.diag(np.sum(weight_matrix, axis=1))
    laplacian = degree_matrix - weight_matrix
    return laplacian
def second_eigenvector(laplacian):
    eigenvalues, eigenvectors = eigh(laplacian)
    return eigenvectors[:, 1]
def spectral_cut(values, weight_matrix, k, sigma):
    W = create_graph(values, k, sigma)
    L = laplacian_matrix(W)
    second_eig = second_eigenvector(L)
    partition = np.sign(second_eig)
    return partition, second_eig
def hierarchical_spectral_clustering(values, k, sigma):
    weight_matrix = create_graph(values, k, sigma)
    linkage_matrix = linkage(values, method='ward')
    return linkage_matrix, weight_matrix
def node_distance(linkage_matrix, point1, point2, num_points):
    node1, node2 = point1, point2
    if node1 < num_points and node2 < num_points:
        return 1
    while node1 >= num_points or node2 >= num_points:
        if node1 >= num_points:
            node1 = int(linkage_matrix[node1 - num_points, 0])
        elif node2 >= num_points:
            node2 = int(linkage_matrix[node2 - num_points, 0])
        else:
            if linkage_matrix[node1 - num_points, 0] == linkage_matrix[node2 - num_points, 0]:
                break
            if linkage_matrix[node1 - num_points, 2] < linkage_matrix[node2 - num_points, 2]:
                node1 = int(linkage_matrix[node1 - num_points, 0])
            else:
                node2 = int(linkage_matrix[node2 - num_points, 0])
    return linkage_matrix[node1 - num_points, 2]
def dasgupta_cost(linkage_matrix, weight_matrix):
    num_points = weight_matrix.shape[0]
    total_cost = 0
    for point1 in range(num_points):
        for point2 in range(point1 + 1, num_points):
            if weight_matrix[point1, point2] > 0:
                lca_dist = node_distance(linkage_matrix, point1, point2, num_points)
                total_cost += weight_matrix[point1, point2] * lca_dist
    return total_cost
k =3
linkage_matrix, weight_matrix = hierarchical_spectral_clustering(values, k, sigma)
cost = dasgupta_cost(linkage_matrix, weight_matrix)
print("dasgupta_cost for k=3",cost)

dasgupta_cost for k=3 309.87232631889486


In [ ]:
#### خوشه بندی سلسله مراتبی طیفی برای k=5

In [141]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import pdist
from scipy.linalg import eigh
from scipy.cluster.hierarchy import linkage, fcluster
link= ["https://cs.joensuu.fi/sipu/datasets/flame.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
values = dataset[["x1", "x2"]].values
distances = pdist(values)
sigma = np.mean(distances)
distances = pdist(values)
sigma = np.mean(distances)
def gaussian_kernel(x, y, sigma):
    return np.exp(-np.linalg.norm(x - y)**2 / (2 * sigma**2))
def create_graph(values, k, sigma):
    num_points = len(values)
    knn = NearestNeighbors(n_neighbors=k)
    knn.fit(values)
    distances, indices = knn.kneighbors(values)
    weight_matrix = np.zeros((num_points, num_points))
    for i in range(num_points):
        for j, idx in zip(range(k), indices[i]):
            if i != idx:
                weight_matrix[i, idx] = gaussian_kernel(values[i], values[idx], sigma)
                weight_matrix[idx, i] = weight_matrix[i, idx]
    return weight_matrix
def laplacian_matrix(weight_matrix):
    degree_matrix = np.diag(np.sum(weight_matrix, axis=1))
    laplacian = degree_matrix - weight_matrix
    return laplacian
def second_eigenvector(laplacian):
    eigenvalues, eigenvectors = eigh(laplacian)
    return eigenvectors[:, 1]
def spectral_cut(values, weight_matrix, k, sigma):
    W = create_graph(values, k, sigma)
    L = laplacian_matrix(W)
    second_eig = second_eigenvector(L)
    partition = np.sign(second_eig)
    return partition, second_eig
def hierarchical_spectral_clustering(values, k, sigma):
    weight_matrix = create_graph(values, k, sigma)
    linkage_matrix = linkage(values, method='ward')
    return linkage_matrix, weight_matrix
def node_distance(linkage_matrix, point1, point2, num_points):
    node1, node2 = point1, point2
    if node1 < num_points and node2 < num_points:
        return 1
    while node1 >= num_points or node2 >= num_points:
        if node1 >= num_points:
            node1 = int(linkage_matrix[node1 - num_points, 0])
        elif node2 >= num_points:
            node2 = int(linkage_matrix[node2 - num_points, 0])
        else:
            if linkage_matrix[node1 - num_points, 0] == linkage_matrix[node2 - num_points, 0]:
                break
            if linkage_matrix[node1 - num_points, 2] < linkage_matrix[node2 - num_points, 2]:
                node1 = int(linkage_matrix[node1 - num_points, 0])
            else:
                node2 = int(linkage_matrix[node2 - num_points, 0])
    return linkage_matrix[node1 - num_points, 2]
def dasgupta_cost(linkage_matrix, weight_matrix):
    num_points = weight_matrix.shape[0]
    total_cost = 0
    for point1 in range(num_points):
        for point2 in range(point1 + 1, num_points):
            if weight_matrix[point1, point2] > 0:
                lca_dist = node_distance(linkage_matrix, point1, point2, num_points)
                total_cost += weight_matrix[point1, point2] * lca_dist
    return total_cost
k =5
linkage_matrix, weight_matrix = hierarchical_spectral_clustering(values, k, sigma)
cost = dasgupta_cost(linkage_matrix, weight_matrix)
print("dasgupta_cost for k=5",cost)

dasgupta_cost for k=5 561.719775069958


In [ ]:
#### خوشه بندی سلسله مراتبی طیفی برای k=10

In [142]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import pdist
from scipy.linalg import eigh
from scipy.cluster.hierarchy import linkage, fcluster
link= ["https://cs.joensuu.fi/sipu/datasets/flame.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
values = dataset[["x1", "x2"]].values
distances = pdist(values)
sigma = np.mean(distances)
distances = pdist(values)
sigma = np.mean(distances)
def gaussian_kernel(x, y, sigma):
    return np.exp(-np.linalg.norm(x - y)**2 / (2 * sigma**2))
def create_graph(values, k, sigma):
    num_points = len(values)
    knn = NearestNeighbors(n_neighbors=k)
    knn.fit(values)
    distances, indices = knn.kneighbors(values)
    weight_matrix = np.zeros((num_points, num_points))
    for i in range(num_points):
        for j, idx in zip(range(k), indices[i]):
            if i != idx:
                weight_matrix[i, idx] = gaussian_kernel(values[i], values[idx], sigma)
                weight_matrix[idx, i] = weight_matrix[i, idx]
    return weight_matrix
def laplacian_matrix(weight_matrix):
    degree_matrix = np.diag(np.sum(weight_matrix, axis=1))
    laplacian = degree_matrix - weight_matrix
    return laplacian
def second_eigenvector(laplacian):
    eigenvalues, eigenvectors = eigh(laplacian)
    return eigenvectors[:, 1]
def spectral_cut(values, weight_matrix, k, sigma):
    W = create_graph(values, k, sigma)
    L = laplacian_matrix(W)
    second_eig = second_eigenvector(L)
    partition = np.sign(second_eig)
    return partition, second_eig
def hierarchical_spectral_clustering(values, k, sigma):
    weight_matrix = create_graph(values, k, sigma)
    linkage_matrix = linkage(values, method='ward')
    return linkage_matrix, weight_matrix
def node_distance(linkage_matrix, point1, point2, num_points):
    node1, node2 = point1, point2
    if node1 < num_points and node2 < num_points:
        return 1
    while node1 >= num_points or node2 >= num_points:
        if node1 >= num_points:
            node1 = int(linkage_matrix[node1 - num_points, 0])
        elif node2 >= num_points:
            node2 = int(linkage_matrix[node2 - num_points, 0])
        else:
            if linkage_matrix[node1 - num_points, 0] == linkage_matrix[node2 - num_points, 0]:
                break
            if linkage_matrix[node1 - num_points, 2] < linkage_matrix[node2 - num_points, 2]:
                node1 = int(linkage_matrix[node1 - num_points, 0])
            else:
                node2 = int(linkage_matrix[node2 - num_points, 0])
    return linkage_matrix[node1 - num_points, 2]
def dasgupta_cost(linkage_matrix, weight_matrix):
    num_points = weight_matrix.shape[0]
    total_cost = 0
    for point1 in range(num_points):
        for point2 in range(point1 + 1, num_points):
            if weight_matrix[point1, point2] > 0:
                lca_dist = node_distance(linkage_matrix, point1, point2, num_points)
                total_cost += weight_matrix[point1, point2] * lca_dist
    return total_cost
k =10
linkage_matrix, weight_matrix = hierarchical_spectral_clustering(values, k, sigma)
cost = dasgupta_cost(linkage_matrix, weight_matrix)
print("dasgupta_cost for k=10",cost)

dasgupta_cost for k=10 1250.8872677337697


In [ ]:
##### جدول مقایسه

In [143]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import AgglomerativeClustering
from scipy.spatial.distance import pdist
from scipy.linalg import eigh
from scipy.cluster.hierarchy import linkage
link = ["https://cs.joensuu.fi/sipu/datasets/flame.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
values = dataset[["x1", "x2"]].values
distances = pdist(values)
sigma = np.mean(distances)
def gaussian_kernel(x, y, sigma):
    return np.exp(-np.linalg.norm(x - y)**2 / (2 * sigma**2))
def create_spectral_graph(values, k, sigma):
    num_points = len(values)
    knn = NearestNeighbors(n_neighbors=k)
    knn.fit(values)
    distances, indices = knn.kneighbors(values)
    weight_matrix = np.zeros((num_points, num_points))
    for i in range(num_points):
        for j, idx in zip(range(k), indices[i]):
            if i != idx:
                weight_matrix[i, idx] = gaussian_kernel(values[i], values[idx], sigma)
                weight_matrix[idx, i] = weight_matrix[i, idx]
    return weight_matrix
def spectral_laplacian_matrix(weight_matrix):
    degree_matrix = np.diag(np.sum(weight_matrix, axis=1))
    laplacian = degree_matrix - weight_matrix
    return laplacian
def spectral_second_eigenvector(laplacian):
    eigenvalues, eigenvectors = eigh(laplacian)
    return eigenvectors[:, 1]
def spectral_dasgupta_cost(linkage_matrix, weight_matrix):
    num_points = weight_matrix.shape[0]
    total_cost = 0
    for point1 in range(num_points):
        for point2 in range(point1 + 1, num_points):
            if weight_matrix[point1, point2] > 0:
                lca_dist = node_distance(linkage_matrix, point1, point2, num_points)
                total_cost += weight_matrix[point1, point2] * lca_dist
    return total_cost
def create_agglomerative_graph(values, sigma):
    num_points = len(values)
    weight_matrix = np.zeros((num_points, num_points))
    for i in range(num_points):
        for j in range(i + 1, num_points):
            weight_matrix[i, j] = gaussian_kernel(values[i], values[j], sigma)
            weight_matrix[j, i] = weight_matrix[i, j]
    return weight_matrix
agglomerative_clustering = AgglomerativeClustering(n_clusters=None, distance_threshold=1.5, linkage='average')
agglomerative_clustering.fit(values)
dataset["new_labels"] = agglomerative_clustering.labels_
linkage_matrix_agglomerative = agglomerative_clustering.children_
weight_matrix_agglomerative = create_agglomerative_graph(values, sigma)
cost_agglomerative = spectral_dasgupta_cost(linkage_matrix_agglomerative, weight_matrix_agglomerative)
k_values = [3, 5, 10]
cost_spectral_values = []
for k in k_values:
    weight_matrix_spectral = create_spectral_graph(values, k, sigma)
    laplacian_spectral = spectral_laplacian_matrix(weight_matrix_spectral)
    second_eig_spectral = spectral_second_eigenvector(laplacian_spectral)
    linkage_matrix_spectral = linkage(values, method='ward')
    cost_spectral = spectral_dasgupta_cost(linkage_matrix_spectral, weight_matrix_spectral)
    cost_spectral_values.append(cost_spectral)
data = {"Method": ["Spectral Clustering (k=3)", "Spectral Clustering (k=5)", "Spectral Clustering (k=10)", "Agglomerative Clustering"],
    "Dasgupta Cost": cost_spectral_values + [cost_agglomerative]}
df_result = pd.DataFrame(data)
print(df_result)

                       Method  Dasgupta Cost
0   Spectral Clustering (k=3)     309.872326
1   Spectral Clustering (k=5)     561.719775
2  Spectral Clustering (k=10)    1250.887268
3    Agglomerative Clustering   17316.747367


In [ ]:
##### مشاهده میشود که مقدار تابع هزینه در الگوریتم خوشه بندی سلسله مراتبی طیفی به طور چشم گیری از الگوریتم تجمیعی بیشتر است

In [ ]:
########الگوریتم سلسله مراتبی طبفی سریع تر عمل میکند در نتیجه میتوان گفت که این الگوریتم بهینه تر از الگوریتم تجمیعی میباشد